In [1]:
library(tidyr)
library(tidyverse) 
library(knitr)
library(dplyr)
library(gmodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ dplyr   1.1.2
✔ tibble  3.2.1     ✔ stringr 1.4.1
✔ readr   2.1.3     ✔ forcats 1.0.0
✔ purrr   0.3.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


# Data Cleaning/Import

In [2]:
# Read in CSV file
test_data <- read.csv("2023StateOf.csv")

# Remove second row so column names are preserved
test_data <- test_data[-c(1),]

# Change column names
colnames(test_data)[1] <- "Record_ID"
colnames(test_data)[2] <- "email"

# Convert "Unchecked" and empty values to NA in all columns
checked_columns <- colnames(test_data)[-c(1, 2)]  # Get column names excluding Record_ID and email

for (col in checked_columns) {
  test_data[[col]] <- ifelse(test_data[[col]] %in% c("Unchecked", ""), NA, test_data[[col]])
}

# Remove rows with all missing values - remove rows where all values are NA except for "Record_ID", "email", and "Q219" columns
test_data <- test_data[rowSums(is.na(test_data[, 3:844])) != (844 - 3 + 1), ]

# Find rows with duplicate email
duplicate_data <- test_data[duplicated(test_data$email) | duplicated(test_data$email, fromLast = TRUE), ]

# Merge rows with duplicate email, keeping only non-missing values and updating Record_ID
merged_data <- duplicate_data %>%
  group_by(email) %>%
  summarize(across(everything(), ~ ifelse(all(is.na(.)), NA, first(na.omit(.)))), .groups = "drop")

# Combine non-duplicate rows with merged rows
final_data <- test_data %>%
  group_by(email) %>%
  summarize(across(everything(), ~ ifelse(all(is.na(.)), NA, first(na.omit(.)))), .groups = "drop")

# Convert "Checked" to 1
final_data[] <- lapply(final_data, function(x) ifelse(x == "Checked", 1, x))

In [3]:
# preview of cleaned data
head(final_data, 10)

email,Record_ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,⋯,Q810,Q811,Q812,Q813,Q814,Q815,Q816,Q817,Q818,Q819
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
,1136,LBGTQ Health Center/Community Center,NA,Connecticut,6511,Urban,Yes,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
19tyler@gmail.com,422,Non-profit organization,NA,District of Columbia,20005,Urban,No,1,NA,⋯,fear of how far laws will go and stigmatizingp -,Unsure,fear,No,NA,Yes,Yes,gov refuses to use evidenced based data,"limitations on providing harm Reduction services, syringe exchanges and safe injection sites in heavily using areas.",Complete
3sollenberger@gmail.com,1294,Non-profit organization,NA,Pennsylvania,15222,Urban,No,1,1,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
6ap464zdb@mozmail.com,687,Community Based Organization (CBO),NA,New York,14527,Rural,No,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
ASHLEY.HART@FLHEALTH.GOV,794,"Government entity (Local, State, Federal)",NA,Florida,32066,Rural,No,1,1,⋯,NA,Unsure,NA,No,NA,No,NA,NA,"WE ARE A VERY SMALL CHD. WE HAVE A SMALL POPULATION AND IF TESTING IS SOUGHT USUALLY PTS WILL GO TO A LARGER CHD WHERE THEY CAN 1. STAY ANONYMOUS, 2 OBTAIN MEDICATIONS/TREATMENT ON SITE 3. SEE A PROVIDER WITH MORE RESOURCES. WE HAVE ONE PROVIDER BETWEEN 2 FACILITIES WHO IS AT OUR LARGER FACILITY 4 DAYS A WEEK AND 2.5 DAYS A MONTH AT OUR SMALL FACILITY. WE ARE CURENTLY SEEKING ANOTHER PROVIDER AS OUR CURRENT ONE IS RETIRING.",Complete
ASIMS@ABSOLUTECARE.COM,1188,Private hospital/clinic setting,NA,Louisiana,70130,Urban,Yes,1,NA,⋯,NA,Unsure,NA,No,NA,No,NA,NA,NA,Complete
Adam.lake@pennmedicine.upenn.edu,901,Academic hospital/clinic setting,NA,Pennsylvania,17602,Suburban,Yes,1,1,⋯,Worse mental health in trans community,Yes,Not sure where to start,No,NA,Yes,Yes,I haven't been able to provide colocated services or openly partner with the harm reduction coalition due to my health system's concern over legality,NA,Complete
Admin@connectdcare.com,123,Private practice,NA,Delaware,19901,Rural,No,1,1,⋯,Yes,Yes,It has,Yes,NA,Yes,Yes,It does,NA,Complete
Aevans@tulane.edu,473,Academic hospital/clinic setting,NA,Louisiana,71301,Suburban,Yes,1,1,⋯,NA,Unsure,NA,No,NA,No,NA,NA,NA,Complete


## Number of incomplete and complete responses

In [4]:
# Complete vs incomplete for general data
table(final_data$Q819)


  Complete Incomplete 
       819        295 

# Descriptive Statistics for overall data

## Age counts & percentages

In [5]:
# Age
gen_age <- table(final_data$Q162)
gen_age
gen_age_perc <- prop.table(gen_age)*100
gen_age_perc


                18-24                 25-34                 35-44 
                   31                   204                   256 
                45-54                 55-64                    65 
                  262                   226                   103 
Prefer not to respond 
                   23 


                18-24                 25-34                 35-44 
             2.805430             18.461538             23.167421 
                45-54                 55-64                    65 
            23.710407             20.452489              9.321267 
Prefer not to respond 
             2.081448 

## Race counts & percentages

In [6]:
## Race
gen_race <- table(final_data$Q159)
gen_race
# Race
gen_race_perc <- prop.table(gen_race)*100
gen_race_perc


        Asian and/or Asian American           Black or African American 
                                 28                                 273 
                        Multiracial    Native American or Alaska Native 
                                 79                                  13 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                                  5                                  55 
            Prefer to self-describe                  White or Caucasian 
                                 26                                 625 


        Asian and/or Asian American           Black or African American 
                          2.5362319                          24.7282609 
                        Multiracial    Native American or Alaska Native 
                          7.1557971                           1.1775362 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                          0.4528986                           4.9818841 
            Prefer to self-describe                  White or Caucasian 
                          2.3550725                          56.6123188 

## Race percentages

In [7]:
# Race
gen_race_perc <- prop.table(gen_race)*100
gen_race_perc


        Asian and/or Asian American           Black or African American 
                          2.5362319                          24.7282609 
                        Multiracial    Native American or Alaska Native 
                          7.1557971                           1.1775362 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                          0.4528986                           4.9818841 
            Prefer to self-describe                  White or Caucasian 
                          2.3550725                          56.6123188 

## Hispanic/Latinx counts

In [8]:
## Hispanic/Latinx
gen_hisp <- table(final_data$Q161)
gen_hisp



                   No Prefer not to respond                   Yes 
                  887                    21                   197 

## Hispanic/Latinx percentages

In [9]:
gen_hisp_perc <- prop.table(gen_hisp)*100
gen_hisp_perc


                   No Prefer not to respond                   Yes 
            80.271493              1.900452             17.828054 

## Gender identity counts

In [10]:
genid_counts <- table(final_data$Q157)
genid_counts


                                Female/Woman 
                                         682 
Genderqueer/gender non-confirming/non-binary 
                                          71 
                                    Male/Man 
                                         288 
                       Prefer not to respond 
                                          19 
                     Prefer to self-describe 
                                          11 
                             Transgender man 
                                          20 
                           Transgender woman 
                                          14 

## Gender identity percentages

In [11]:
genid_perc <- prop.table(genid_counts)*100
genid_perc


                                Female/Woman 
                                  61.7194570 
Genderqueer/gender non-confirming/non-binary 
                                   6.4253394 
                                    Male/Man 
                                  26.0633484 
                       Prefer not to respond 
                                   1.7194570 
                     Prefer to self-describe 
                                   0.9954751 
                             Transgender man 
                                   1.8099548 
                           Transgender woman 
                                   1.2669683 

## Type of organization counts

In [12]:
## Org type
gen_org_type <- table(final_data$Q1)
gen_org_type


             Academic hospital/clinic setting 
                                           80 
                        Advocacy organization 
                                            5 
              AIDS Service Organization (ASO) 
                                          133 
           Community Based Organization (CBO) 
                                          129 
     Federally Qualified Health Center (FQHC) 
                                          134 
    Government entity (Local, State, Federal) 
                                          105 
               Health department (non-clinic) 
                                           41 
Health department clinic/public health clinic 
                                          146 
         LBGTQ Health Center/Community Center 
                                           55 
                      Non-profit organization 
                                          159 
                                        Other 
            

## Type of organization percentages

In [13]:
gen_org_perc <- prop.table(gen_org_type)*100
gen_org_perc


             Academic hospital/clinic setting 
                                    7.1813285 
                        Advocacy organization 
                                    0.4488330 
              AIDS Service Organization (ASO) 
                                   11.9389587 
           Community Based Organization (CBO) 
                                   11.5798923 
     Federally Qualified Health Center (FQHC) 
                                   12.0287253 
    Government entity (Local, State, Federal) 
                                    9.4254937 
               Health department (non-clinic) 
                                    3.6804309 
Health department clinic/public health clinic 
                                   13.1059246 
         LBGTQ Health Center/Community Center 
                                    4.9371634 
                      Non-profit organization 
                                   14.2728905 
                                        Other 
            

## State counts

In [14]:
## States
gen_state_counts <- table(final_data$Q3)
gen_state_counts


             Alabama               Alaska              Arizona 
                   8                    1                   15 
            Arkansas           California             Colorado 
                   8                  105                   16 
         Connecticut             Delaware District of Columbia 
                  20                   12                   39 
             Florida              Georgia               Hawaii 
                  64                   26                   11 
               Idaho             Illinois              Indiana 
                   2                   38                   46 
                Iowa               Kansas             Kentucky 
                  13                    4                   30 
           Louisiana                Maine             Maryland 
                  24                    1                   67 
       Massachusetts             Michigan            Minnesota 
                   7                   

## State percentages

In [15]:
gen_state_perc <- prop.table(gen_state_counts)*100
gen_state_perc


             Alabama               Alaska              Arizona 
          0.72072072           0.09009009           1.35135135 
            Arkansas           California             Colorado 
          0.72072072           9.45945946           1.44144144 
         Connecticut             Delaware District of Columbia 
          1.80180180           1.08108108           3.51351351 
             Florida              Georgia               Hawaii 
          5.76576577           2.34234234           0.99099099 
               Idaho             Illinois              Indiana 
          0.18018018           3.42342342           4.14414414 
                Iowa               Kansas             Kentucky 
          1.17117117           0.36036036           2.70270270 
           Louisiana                Maine             Maryland 
          2.16216216           0.09009009           6.03603604 
       Massachusetts             Michigan            Minnesota 
          0.63063063           3.873873

## Geographic area counts

In [16]:
geog_counts <- table(final_data$Q5)
geog_counts


   Rural Suburban   Tribal    Urban 
     228      195        3      686 

## Geographic area percentages

In [17]:
geog_perc <- prop.table(geog_counts)*100
geog_perc


     Rural   Suburban     Tribal      Urban 
20.5035971 17.5359712  0.2697842 61.6906475 

## Role type counts - Clinical vs nonclinical

In [18]:
role_type_counts <- table(final_data$Q65)
role_type_counts


    Clinical Non-clinical 
         380          728 

In [19]:
role_type_perc <- prop.table(role_type_counts)*100
role_type_perc


    Clinical Non-clinical 
    34.29603     65.70397 

## Clinical role counts

In [20]:
clinic_role_counts <- table(final_data$Q66)
clinic_role_counts


                             Dentist               Dietician/Nutritionist 
                                   2                                    1 
     Disease Intervention Specialist      Licensed Clinical Social Worker 
                                   1                                   50 
            Licensed Practical Nurse                    Medical Assistant 
                                  10                                   10 
          Mental Health Case Manager              Mental Health Counselor 
                                   5                                    1 
          Mental Health Professional                        Nurse Manager 
                                  16                                   19 
                       Nurse Midwife                   Nurse Practitioner 
                                   1                                   69 
Other Provider/Clinical professional                           Pharmacist 
                        

## Clinical role percentages

In [21]:
clinic_role_perc <- prop.table(clinic_role_counts) * 100 
clinic_role_perc


                             Dentist               Dietician/Nutritionist 
                           0.5277045                            0.2638522 
     Disease Intervention Specialist      Licensed Clinical Social Worker 
                           0.2638522                           13.1926121 
            Licensed Practical Nurse                    Medical Assistant 
                           2.6385224                            2.6385224 
          Mental Health Case Manager              Mental Health Counselor 
                           1.3192612                            0.2638522 
          Mental Health Professional                        Nurse Manager 
                           4.2216359                            5.0131926 
                       Nurse Midwife                   Nurse Practitioner 
                           0.2638522                           18.2058047 
Other Provider/Clinical professional                           Pharmacist 
                        

## Non-clinical role counts 

In [22]:
nonclinic_role_counts <- table(final_data$Q71)
nonclinic_role_counts


                         Administrator                               Advocate 
                                   169                                     18 
                             Caregiver      Case Manager/Medical case manager 
                                     1                                    129 
       Clergy/Faith-Based Professional                Community Health Worker 
                                     1                                     64 
       Disease Intervention Specialist                 Finance/Fiscal Manager 
                                    37                                      4 
   Harm Reductionist/Risk Reductionist              Health and Wellness Coach 
                                    32                                      1 
           Health Education Specialist                       Health Navigator 
                                    56                                     36 
               HIV Linkage Coordinator             

## Non-clinical role percentages 

In [23]:
nonclinic_role_perc <- prop.table(nonclinic_role_counts)*100
nonclinic_role_perc


                         Administrator                               Advocate 
                            23.2782369                              2.4793388 
                             Caregiver      Case Manager/Medical case manager 
                             0.1377410                             17.7685950 
       Clergy/Faith-Based Professional                Community Health Worker 
                             0.1377410                              8.8154270 
       Disease Intervention Specialist                 Finance/Fiscal Manager 
                             5.0964187                              0.5509642 
   Harm Reductionist/Risk Reductionist              Health and Wellness Coach 
                             4.4077135                              0.1377410 
           Health Education Specialist                       Health Navigator 
                             7.7134986                              4.9586777 
               HIV Linkage Coordinator             

## Years in role counts

In [24]:
years_counts <- table(final_data$Q94)
years_counts


   <2 years 11-20 years   2-4 years   21+ years  5-10 years 
        292         166         237         178         235 

## Years in role percentages

In [25]:
years_perc <- prop.table(years_counts)*100
years_perc


   <2 years 11-20 years   2-4 years   21+ years  5-10 years 
   26.35379    14.98195    21.38989    16.06498    21.20939 

## Prescriber counts

In [26]:
prescriber_counts <- table(final_data$Q70)
prescriber_counts


 No Yes 
229 154 

## Prescriber percentage

In [27]:
prescriber_perc <- prop.table(prescriber_counts)*100
prescriber_perc


      No      Yes 
59.79112 40.20888 

## Onsite pharmacy counts

In [28]:
onsite_pharm_counts <- table(final_data$Q6)
onsite_pharm_counts


 No Yes 
681 430 

## Onsite pharmacy percentages

In [29]:
onsite_pharm_perc <- prop.table(onsite_pharm_counts)*100
onsite_pharm_perc


      No      Yes 
61.29613 38.70387 

# HIV training statistics for overall data

In [30]:
# Create a vector of column names representing the HIV topics columns
hiv_topics <- c("Q198", "Q199", "Q200", "Q201", "Q202","Q203","Q204","Q205","Q206","Q207","Q208","Q209","Q210a","Q210b")

## HIV topic with greatest number of 1st rankings

In [31]:
# Calculate the counts of 1st rankings for each HIV topic
hiv1_counts <- sapply(hiv_topics, function(topic) sum(final_data[[topic]] == 1, na.rm = TRUE))
hiv1_counts

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  181    83    76    18    21    37    34    49    49     6    12    12     3 
Q210b 
    1

In [32]:
# Find the HIV topic with the highest count of 1st rankings
hiv1 <- hiv_topics[which.max(hiv1_counts)]
hiv1

[1] "Q198"

In [33]:
# Calculate the percentage of 1st rankings
hiv1_perc <- (hiv1_counts[which.max(hiv1_counts)] / sum(rowSums(!is.na(final_data[, hiv_topics])) > 0)) * 100
hiv1_perc

Q198 
30.83475

## HIV topic with greatest number of 2nd rankings

In [34]:
# Calculate the counts of 2nd rankings for each HIV topic
hiv2_counts <- sapply(hiv_topics, function(topic) sum(final_data[[topic]] == 2, na.rm = TRUE))
hiv2_counts

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   61    89    76    27    44    52    48    49    56    15    19     2     0 
Q210b 
    1

In [35]:
# Find the HIV topic with the highest count of 2nd rankings
hiv2 <- hiv_topics[which.max(hiv2_counts)]
hiv2

[1] "Q199"

In [36]:
# Calculate the percentage of 2nd rankings for the top HIV topic
hiv2_perc <- (hiv2_counts[which.max(hiv2_counts)] / sum(rowSums(!is.na(final_data[, hiv_topics])) > 0)) * 100
hiv2_perc

Q199 
15.16184

## HIV topic with greatest number of 3rd rankings

In [37]:
# Calculate the counts of 3rd rankings for each HIV topic
hiv3_counts <- sapply(hiv_topics, function(topic) sum(final_data[[topic]] == 3, na.rm = TRUE))
hiv3_counts

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   67    47    56    17    56    46    61    58    53    12    37    16     2 
Q210b 
    3

In [38]:
# Find the HIV topic with the highest count of 3rd rankings
hiv3 <- hiv_topics[which.max(hiv3_counts)]
hiv3

[1] "Q198"

In [39]:
# Calculate the percentage of 3rd rankings
hiv3_perc <- (hiv3_counts[which.max(hiv3_counts)] / sum(rowSums(!is.na(final_data[, hiv_topics])) > 0)) * 100
hiv3_perc

Q198 
11.41397

## Top 3 HIV training topics

In [40]:
# Calculate the counts for each LGBTQ topic
hiv_topic_counts <- sapply(hiv_topics, function(topic) sum(final_data[[topic]] %in% c(1, 2, 3), na.rm = TRUE))
hiv_topic_counts
# Sort the topics based on their counts in descending order
hiv_sorted_topics <- sort(hiv_topic_counts, decreasing = TRUE)
# Get the top 3 topics
hiv_top_3_topics <- names(hiv_sorted_topics)[1:3]

hiv_top_3_topics

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  309   219   208    62   121   135   143   156   158    33    68    30     5 
Q210b 
    5

[1] "Q198" "Q199" "Q200"

In [41]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
hiv_total_respondents <- sum(rowSums(!is.na(final_data[, hiv_topics])) > 0)

# Calculate the percentage for the first top3_prep_topics
hiv1_percentage <- (hiv_sorted_topics[1] / hiv_total_respondents) * 100

# Calculate the percentage for the second top3_prep_topics
hiv2_percentage <- (hiv_sorted_topics[2] / hiv_total_respondents) * 100

# Calculate the percentage for the third top3_prep_topics
hiv3_percentage <- (hiv_sorted_topics[3] / hiv_total_respondents) * 100

# Print the results
hiv1_percentage
hiv2_percentage
hiv3_percentage

Q198 
52.64055

Q199 
37.30835

Q200 
35.43441

# PrEP training statistics for overall data

In [42]:
# Create a vector of column names representing the PrEP topics columns
prep_topics <- c("Q211", "Q212", "Q213", "Q214", "Q215","Q216","Q217","Q218","Q219","Q220","Q221","Q222","Q223","Q224","Q225")

## PrEP topic with greatest number of 1st rankings

In [43]:
# Calculate the counts of 1st rankings for each PrEP topic
prep1_counts <- sapply(prep_topics, function(topic) sum(final_data[[topic]] == 1, na.rm = TRUE))
prep1_counts

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
 124   73   62  108   11   22   20   48   37   31   19   29   36   12    3

In [44]:
# Find the PrEP topic with the highest count of 1st rankings
prep1 <- prep_topics[which.max(prep1_counts)]
prep1

[1] "Q211"

In [45]:
# Calculate the percentage of 1st rankings
prep1_perc <- (prep1_counts[which.max(prep1_counts)] / sum(rowSums(!is.na(final_data[, prep_topics])) > 0)) * 100
prep1_perc

Q211 
19.2846

## PrEP topic with greatest number of 2nd rankings

In [46]:
# Calculate the counts of 2nd rankings for each PrEP topic
prep2_counts <- sapply(prep_topics, function(topic) sum(final_data[[topic]] == 2, na.rm = TRUE))
prep2_counts

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  29   70   53   80   26   34   37   77   50   39   24   27   41   15    0

In [47]:
# Find the PrEP topic with the highest count of 2nd rankings
prep2 <- prep_topics[which.max(prep2_counts)]
prep2

[1] "Q214"

In [48]:
# Calculate the percentage of 2nd rankings for the top PrEP topic
prep2_perc <- (prep2_counts[which.max(prep2_counts)] / sum(rowSums(!is.na(final_data[, prep_topics])) > 0)) * 100
prep2_perc

Q214 
12.44168

## PrEP topic with greatest number of third rankings

In [49]:
# Calculate the counts of 3rd rankings for each PrEP topic
prep3_counts <- sapply(prep_topics, function(topic) sum(final_data[[topic]] == 3, na.rm = TRUE))
prep3_counts

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  44   21   43   52   18   35   38   69   26   66   42   46   42   41    5

In [50]:
# Find the PrEP topic with the highest count of 3rd rankings
prep3 <- prep_topics[which.max(prep3_counts)]
prep3

[1] "Q218"

In [51]:
# Calculate the percentage of 3rd rankings
prep3_perc <- (prep3_counts[which.max(prep3_counts)] / sum(rowSums(!is.na(final_data[, prep_topics])) > 0)) * 100
prep3_perc

Q218 
10.73095

## Top 3 PrEP training topics

In [52]:
# Calculate the counts for each PrEP topic
prep_topic_counts <- sapply(prep_topics, function(topic) sum(final_data[[topic]] %in% c(1, 2, 3), na.rm = TRUE))
prep_topic_counts
# Sort the topics based on their counts in descending order
prep_sorted_topics <- sort(prep_topic_counts, decreasing = TRUE)

# Get the top 3 topics
prep_top_3_topics <- names(prep_sorted_topics)[1:3]

prep_top_3_topics

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
 197  164  158  240   55   91   95  194  113  136   85  102  119   68    8

[1] "Q214" "Q211" "Q218"

In [53]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the PrEP topic questions
prep_total_respondents <- sum(rowSums(!is.na(final_data[, prep_topics])) > 0)

# Calculate the percentage for the first top3_prep_topics
prep1_percentage <- (prep_sorted_topics[1] / prep_total_respondents) * 100

# Calculate the percentage for the second top3_prep_topics
prep2_percentage <- (prep_sorted_topics[2] / prep_total_respondents) * 100

# Calculate the percentage for the third top3_prep_topics
prep3_percentage <- (prep_sorted_topics[3] / prep_total_respondents) * 100

# Print the results
prep1_percentage
prep2_percentage
prep3_percentage

Q214 
37.32504

Q211 
30.63764

Q218 
30.17107

# LGBTQ training statistics for overall data

In [54]:
# Create a vector of column names representing the LGBTQ topics columns
lgbtq_topics <- c("Q227", "Q228", "Q229", "Q230", "Q231","Q232","Q233","Q234","Q235")

## LGBTQ topic with greatest number of 1st rankings

In [55]:
# Calculate the counts of 1st rankings for each LGBTQ topic
lgbtq1_counts <- sapply(lgbtq_topics, function(topic) sum(final_data[[topic]] == 1, na.rm = TRUE))
lgbtq1_counts

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
 139  198  121  120   34   67  118  114   15

In [56]:
# Find the LGBTQ topic with the highest count of 1st rankings
lgbtq1 <- lgbtq_topics[which.max(lgbtq1_counts)]
lgbtq1

[1] "Q228"

In [57]:
# Calculate the percentage of 1st rankings
lgbtq1_perc <- (lgbtq1_counts[which.max(lgbtq1_counts)] / sum(rowSums(!is.na(final_data[, lgbtq_topics])) > 0)) * 100
lgbtq1_perc

Q228 
21.10874

## LGBTQ topic with greatest number of 2nd rankings

In [58]:
# Calculate the counts of 2nd rankings for each LGBTQ topic
lgbtq2_counts <- sapply(lgbtq_topics, function(topic) sum(final_data[[topic]] == 2, na.rm = TRUE))
lgbtq2_counts

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  67  121  102  154   62   78  143  143    4

In [59]:
# Find the LGBTQ topic with the highest count of 2nd rankings
lgbtq2 <- lgbtq_topics[which.max(lgbtq2_counts)]
lgbtq2

[1] "Q230"

In [60]:
# Calculate the percentage of 2nd rankings for the top LGBTQ topic
lgbtq2_perc <- (lgbtq2_counts[which.max(lgbtq2_counts)] / sum(rowSums(!is.na(final_data[, lgbtq_topics])) > 0)) * 100
lgbtq2_perc

Q230 
16.41791

## LGBTQ topic with greatest number of third rankings

In [61]:
# Calculate the counts of 3rd rankings for each LGBTQ topic
lgbtq3_counts <- sapply(lgbtq_topics, function(topic) sum(final_data[[topic]] == 3, na.rm = TRUE))
lgbtq3_counts

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  81  122   73  124   66   64  109  194   14

In [62]:
# Find the LGBTQ topic with the highest count of 3rd rankings
lgbtq3 <- lgbtq_topics[which.max(lgbtq3_counts)]
lgbtq3

[1] "Q234"

In [63]:
# Calculate the percentage of 3rd rankings
lgbtq3_perc <- (lgbtq3_counts[which.max(lgbtq3_counts)] / sum(rowSums(!is.na(final_data[, lgbtq_topics])) > 0)) * 100
lgbtq3_perc

Q234 
20.6823

## Top 3 LGBTQ training topics 

In [64]:
# Calculate the counts for each LGBTQ topic
lgbtq_topic_counts <- sapply(lgbtq_topics, function(topic) sum(final_data[[topic]] %in% c(1, 2, 3), na.rm = TRUE))
lgbtq_topic_counts
# Sort the topics based on their counts in descending order
lgbtq_sorted_topics <- sort(lgbtq_topic_counts, decreasing = TRUE)

# Get the top 3 topics
lgbtq_top_3_topics <- names(lgbtq_sorted_topics)[1:3]

lgbtq_top_3_topics

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
 287  441  296  398  162  209  370  451   33

[1] "Q234" "Q228" "Q230"

In [65]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the LGBTQ topic questions
lgtbq_total_respondents <- sum(rowSums(!is.na(final_data[, lgbtq_topics])) > 0)

# Calculate the percentage for the first top3_lgbtq_topics
lgbtq1_percentage <- (lgbtq_sorted_topics[1] / lgtbq_total_respondents) * 100

# Calculate the percentage for the second top3_lgbtq_topics
lgbtq2_percentage <- (lgbtq_sorted_topics[2] / lgtbq_total_respondents) * 100

# Calculate the percentage for the third top3_lgbtq_topics
lgbtq3_percentage <- (lgbtq_sorted_topics[3] / lgtbq_total_respondents) * 100

# Print the results
lgbtq1_percentage
lgbtq2_percentage
lgbtq3_percentage

Q234 
48.08102

Q228 
47.01493

Q230 
42.4307

# Identifying HIV providers

In [66]:
# HIV providers
HIV_providers <- final_data[final_data$Q22 == "Yes" | final_data$Q30 == "Yes" | final_data$Q32 == "Yes", ]
head(HIV_providers, 10)

email,Record_ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,⋯,Q810,Q811,Q812,Q813,Q814,Q815,Q816,Q817,Q818,Q819
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
19tyler@gmail.com,422,Non-profit organization,NA,District of Columbia,20005,Urban,No,1,NA,⋯,fear of how far laws will go and stigmatizingp -,Unsure,fear,No,NA,Yes,Yes,gov refuses to use evidenced based data,"limitations on providing harm Reduction services, syringe exchanges and safe injection sites in heavily using areas.",Complete
3sollenberger@gmail.com,1294,Non-profit organization,NA,Pennsylvania,15222,Urban,No,1,1,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
ASHLEY.HART@FLHEALTH.GOV,794,"Government entity (Local, State, Federal)",NA,Florida,32066,Rural,No,1,1,⋯,NA,Unsure,NA,No,NA,No,NA,NA,"WE ARE A VERY SMALL CHD. WE HAVE A SMALL POPULATION AND IF TESTING IS SOUGHT USUALLY PTS WILL GO TO A LARGER CHD WHERE THEY CAN 1. STAY ANONYMOUS, 2 OBTAIN MEDICATIONS/TREATMENT ON SITE 3. SEE A PROVIDER WITH MORE RESOURCES. WE HAVE ONE PROVIDER BETWEEN 2 FACILITIES WHO IS AT OUR LARGER FACILITY 4 DAYS A WEEK AND 2.5 DAYS A MONTH AT OUR SMALL FACILITY. WE ARE CURENTLY SEEKING ANOTHER PROVIDER AS OUR CURRENT ONE IS RETIRING.",Complete
ASIMS@ABSOLUTECARE.COM,1188,Private hospital/clinic setting,NA,Louisiana,70130,Urban,Yes,1,NA,⋯,NA,Unsure,NA,No,NA,No,NA,NA,NA,Complete
Adam.lake@pennmedicine.upenn.edu,901,Academic hospital/clinic setting,NA,Pennsylvania,17602,Suburban,Yes,1,1,⋯,Worse mental health in trans community,Yes,Not sure where to start,No,NA,Yes,Yes,I haven't been able to provide colocated services or openly partner with the harm reduction coalition due to my health system's concern over legality,NA,Complete
Admin@connectdcare.com,123,Private practice,NA,Delaware,19901,Rural,No,1,1,⋯,Yes,Yes,It has,Yes,NA,Yes,Yes,It does,NA,Complete
Aevans@tulane.edu,473,Academic hospital/clinic setting,NA,Louisiana,71301,Suburban,Yes,1,1,⋯,NA,Unsure,NA,No,NA,No,NA,NA,NA,Complete
Agirmay@kintegra.org,388,Federally Qualified Health Center (FQHC),NA,North Carolina,28052,Suburban,Yes,1,1,⋯,NA,Yes,It is not right to deny healthcare because of their gender,No,NA,No,NA,NA,No,Complete


## Number of incomplete and complete responses among HIV providers

In [67]:
# Complete vs Incomplete among HIV providers
table(HIV_providers$Q819)


  Complete Incomplete 
       760        271 

## Descriptive Statistics among HIV providers

### Age counts among HIV providers

In [68]:
## HIV providers - Age
age_HIV <- table(HIV_providers$Q162)
age_HIV


                18-24                 25-34                 35-44 
                   31                   196                   239 
                45-54                 55-64                    65 
                  249                   204                    91 
Prefer not to respond 
                   19 

### Age percentages among HIV providers

In [69]:
age_perc_HIV <- prop.table(age_HIV)*100
age_perc_HIV


                18-24                 25-34                 35-44 
             3.012634             19.047619             23.226433 
                45-54                 55-64                    65 
            24.198251             19.825073              8.843537 
Prefer not to respond 
             1.846453 

### Gender counts among HIV providers

In [70]:
## HIV providers - Gender
gender_HIV<- table(HIV_providers$Q157)
gender_HIV


                                Female/Woman 
                                         636 
Genderqueer/gender non-confirming/non-binary 
                                          64 
                                    Male/Man 
                                         271 
                       Prefer not to respond 
                                          17 
                     Prefer to self-describe 
                                           9 
                             Transgender man 
                                          18 
                           Transgender woman 
                                          14 

### Gender percentages among HIV providers

In [71]:
gender_perc_HIV <- prop.table(gender_HIV)*100
gender_perc_HIV


                                Female/Woman 
                                  61.8075802 
Genderqueer/gender non-confirming/non-binary 
                                   6.2196307 
                                    Male/Man 
                                  26.3362488 
                       Prefer not to respond 
                                   1.6520894 
                     Prefer to self-describe 
                                   0.8746356 
                             Transgender man 
                                   1.7492711 
                           Transgender woman 
                                   1.3605442 

### Race counts among HIV providers

In [72]:
## HIV providers - Race
race_HIV <- table(HIV_providers$Q159)
race_HIV


        Asian and/or Asian American           Black or African American 
                                 26                                 250 
                        Multiracial    Native American or Alaska Native 
                                 77                                  11 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                                  5                                  52 
            Prefer to self-describe                  White or Caucasian 
                                 25                                 582 

### Race percentages among HIV providers

In [73]:
race_perc_HIV <- prop.table(race_HIV)*100
race_perc_HIV


        Asian and/or Asian American           Black or African American 
                          2.5291829                          24.3190661 
                        Multiracial    Native American or Alaska Native 
                          7.4902724                           1.0700389 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                          0.4863813                           5.0583658 
            Prefer to self-describe                  White or Caucasian 
                          2.4319066                          56.6147860 

### Hispanic/Latinx counts among HIV providers

In [74]:
## HIV providers - Hispanic/Latinx
hisp_HIV <- table(HIV_providers$Q161)
hisp_HIV


                   No Prefer not to respond                   Yes 
                  819                    19                   191 

### Hispanic/Latinx percentages among HIV providers

In [75]:
hisp_perc_HIV<- prop.table(hisp_HIV)*100
hisp_perc_HIV


                   No Prefer not to respond                   Yes 
            79.591837              1.846453             18.561710 

### Geographic counts among HIV providers

In [76]:
geog_counts_HIV <- table(HIV_providers$Q5)
geog_counts_HIV 


   Rural Suburban   Tribal    Urban 
     215      171        3      642 

### Geographic percentages among HIV providers

In [77]:
geog_perc_HIV <- prop.table(geog_counts_HIV)*100
geog_perc_HIV


     Rural   Suburban     Tribal      Urban 
20.8535403 16.5858390  0.2909796 62.2696411 

### Setting counts among HIV providers

In [78]:
setting_counts_HIV <- table(HIV_providers$Q1)
setting_counts_HIV


             Academic hospital/clinic setting 
                                           76 
                        Advocacy organization 
                                            4 
              AIDS Service Organization (ASO) 
                                          132 
           Community Based Organization (CBO) 
                                          111 
     Federally Qualified Health Center (FQHC) 
                                          134 
    Government entity (Local, State, Federal) 
                                           95 
               Health department (non-clinic) 
                                           39 
Health department clinic/public health clinic 
                                          144 
         LBGTQ Health Center/Community Center 
                                           51 
                      Non-profit organization 
                                          140 
                                        Other 
            

### Setting percentages among HIV providers

In [79]:
setting_perc_HIV <- prop.table(setting_counts_HIV)*100
setting_perc_HIV


             Academic hospital/clinic setting 
                                    7.3714840 
                        Advocacy organization 
                                    0.3879728 
              AIDS Service Organization (ASO) 
                                   12.8031038 
           Community Based Organization (CBO) 
                                   10.7662464 
     Federally Qualified Health Center (FQHC) 
                                   12.9970902 
    Government entity (Local, State, Federal) 
                                    9.2143550 
               Health department (non-clinic) 
                                    3.7827352 
Health department clinic/public health clinic 
                                   13.9670223 
         LBGTQ Health Center/Community Center 
                                    4.9466537 
                      Non-profit organization 
                                   13.5790495 
                                        Other 
            

### Role type counts among HIV providers - clinical vs non-clinical

In [80]:
role_type_counts_HIV <- table(HIV_providers$Q65)
role_type_counts_HIV


    Clinical Non-clinical 
         344          686 

### Role type percentages among HIV providers - clinical vs non-clinical

In [81]:
role_type_perc_HIV <- prop.table(role_type_counts_HIV)*100
role_type_perc_HIV


    Clinical Non-clinical 
    33.39806     66.60194 

### Clinical role counts among HIV providers

In [82]:
role_counts_HIV <- table(HIV_providers$Q66)
role_counts_HIV


                             Dentist      Disease Intervention Specialist 
                                   2                                    1 
     Licensed Clinical Social Worker             Licensed Practical Nurse 
                                  37                                   10 
                   Medical Assistant           Mental Health Case Manager 
                                  10                                    3 
             Mental Health Counselor           Mental Health Professional 
                                   1                                   12 
                       Nurse Manager                        Nurse Midwife 
                                  18                                    1 
                  Nurse Practitioner Other Provider/Clinical professional 
                                  68                                    7 
                          Pharmacist                  Pharmacy Technician 
                        

### Clinical role percentages among HIV providers

In [83]:
role_perc_HIV <- prop.table(role_counts_HIV)*100
role_perc_HIV


                             Dentist      Disease Intervention Specialist 
                           0.5830904                            0.2915452 
     Licensed Clinical Social Worker             Licensed Practical Nurse 
                          10.7871720                            2.9154519 
                   Medical Assistant           Mental Health Case Manager 
                           2.9154519                            0.8746356 
             Mental Health Counselor           Mental Health Professional 
                           0.2915452                            3.4985423 
                       Nurse Manager                        Nurse Midwife 
                           5.2478134                            0.2915452 
                  Nurse Practitioner Other Provider/Clinical professional 
                          19.8250729                            2.0408163 
                          Pharmacist                  Pharmacy Technician 
                        

### Non-clinical role counts among HIV providers

In [84]:
nonclinic_role_counts_HIV <- table(HIV_providers$Q71)
nonclinic_role_counts_HIV


                         Administrator                               Advocate 
                                   157                                     15 
     Case Manager/Medical case manager        Clergy/Faith-Based Professional 
                                   123                                      1 
               Community Health Worker        Disease Intervention Specialist 
                                    64                                     37 
                Finance/Fiscal Manager    Harm Reductionist/Risk Reductionist 
                                     2                                     32 
             Health and Wellness Coach            Health Education Specialist 
                                     1                                     56 
                      Health Navigator                HIV Linkage Coordinator 
                                    35                                      4 
             HIV Retention Coordinator             

### Non-clinical role percentages among HIV providers

In [85]:
nonclinic_role_perc_HIV <- prop.table(nonclinic_role_counts_HIV)*100
nonclinic_role_perc_HIV


                         Administrator                               Advocate 
                            22.9532164                              2.1929825 
     Case Manager/Medical case manager        Clergy/Faith-Based Professional 
                            17.9824561                              0.1461988 
               Community Health Worker        Disease Intervention Specialist 
                             9.3567251                              5.4093567 
                Finance/Fiscal Manager    Harm Reductionist/Risk Reductionist 
                             0.2923977                              4.6783626 
             Health and Wellness Coach            Health Education Specialist 
                             0.1461988                              8.1871345 
                      Health Navigator                HIV Linkage Coordinator 
                             5.1169591                              0.5847953 
             HIV Retention Coordinator             

### Years in role counts among HIV providers

In [86]:
years_counts_HIV <- table(HIV_providers$Q94)
years_counts_HIV


   <2 years 11-20 years   2-4 years   21+ years  5-10 years 
        285         150         219         157         219 

### Years in role percentages among HIV providers

In [87]:
years_perc_HIV <- prop.table(years_counts_HIV)*100
years_perc_HIV


   <2 years 11-20 years   2-4 years   21+ years  5-10 years 
   27.66990    14.56311    21.26214    15.24272    21.26214 

### Prescriber counts among HIV providers

In [88]:
prescriber_counts_HIV <- table(HIV_providers$Q70)
prescriber_counts_HIV


 No Yes 
198 149 

### Prescriber percentages among HIV providers

In [89]:
prescriber_perc_HIV <- prop.table(prescriber_counts_HIV)*100
prescriber_perc_HIV


      No      Yes 
57.06052 42.93948 

### Onsite pharmacy counts among HIV providers

In [90]:
onsite_pharm_counts_HIV <- table(HIV_providers$Q6)
onsite_pharm_counts_HIV


 No Yes 
613 418 

### Onsite pharmacy percentages among HIV providers

In [91]:
onsite_pharm_perc_HIV <- prop.table(onsite_pharm_counts_HIV)*100
onsite_pharm_perc_HIV


      No      Yes 
59.45684 40.54316 

## HIV training statistics for HIV providers

In [92]:
# HIV providers - Create a vector of column names representing the HIV topics columns
hiv_topics_HIV <- c("Q198", "Q199", "Q200", "Q201", "Q202","Q203","Q204","Q205","Q206","Q207","Q208","Q209","Q210a","Q210b")

### HIV topic with greatest number of 1st rankings among HIV providers

In [93]:
# HIV providers - Calculate the counts of 1st rankings for each HIV topic
hiv1_count_HIV <- sapply(hiv_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 1, na.rm = TRUE))
hiv1_count_HIV

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  181    83    76    18    21    37    34    49    49     6    12    12     3 
Q210b 
    1

In [94]:
# HIV providers - Find the HIV topic with the highest count of 1st rankings
hiv1_rank_HIV <- hiv_topics_HIV[which.max(hiv1_count_HIV)]
hiv1_rank_HIV

[1] "Q198"

In [95]:
# HIV providers - Calculate the percentage of 1st rankings
hiv1_perc_HIV <- (hiv1_count_HIV[which.max(hiv1_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, hiv_topics_HIV])) > 0)) * 100
hiv1_perc_HIV

Q198 
30.83475

### HIV topic with greatest number of 2nd rankings among HIV providers

In [96]:
# HIV providers - Calculate the counts of 2nd rankings for each HIV topic
hiv2_count_HIV <- sapply(hiv_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 2, na.rm = TRUE))
hiv2_count_HIV

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   61    89    76    27    44    52    48    49    56    15    19     2     0 
Q210b 
    1

In [97]:
# HIV providers - Find the HIV topic with the highest count of 2nd rankings
hiv2_rank_HIV <- hiv_topics_HIV[which.max(hiv2_count_HIV)]
hiv2_rank_HIV

[1] "Q199"

In [98]:
# HIV providers - Calculate the percentage of 2nd ranking
hiv2_perc_HIV <- (hiv2_count_HIV[which.max(hiv2_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, hiv_topics_HIV])) > 0)) * 100
hiv2_perc_HIV

Q199 
15.16184

### HIV topic with greatest number of 3rd rankings among HIV providers

In [99]:
# HIV providers - Calculate the counts of 3rd rankings for each HIV topic
hiv3_count_HIV <- sapply(hiv_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 3, na.rm = TRUE))
hiv3_count_HIV

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   67    47    56    17    56    46    61    58    53    12    37    16     2 
Q210b 
    3

In [100]:
# HIV providers - Find the HIV topic with the highest count of 3rd rankings
hiv3_rank_HIV <- hiv_topics_HIV[which.max(hiv3_count_HIV)]
hiv3_rank_HIV

[1] "Q198"

In [101]:
# HIV providers - Calculate the percentage of 3rd rankings
hiv3_perc_HIV <- (hiv3_count_HIV[which.max(hiv3_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, hiv_topics_HIV])) > 0)) * 100
hiv3_perc_HIV

Q198 
11.41397

### Top 3 HIV topics among HIV providers

In [102]:
# Calculate the counts for each HIV topic
hiv_topic_counts_HIV <- sapply(hiv_topics_HIV, function(topic) sum(HIV_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
hiv_sorted_topics_HIV <- sort(hiv_topic_counts_HIV, decreasing = TRUE)

# Get the top 3 topics
top3_hiv_topics_HIV <- names(hiv_sorted_topics_HIV)[1:3]

top3_hiv_topics_HIV

[1] "Q198" "Q199" "Q200"

### Percentage of each top 3 HIV topic among HIV providers

In [103]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
hiv_total_respondents_HIV <- sum(rowSums(!is.na(HIV_providers[, hiv_topics_HIV])) > 0)

# Calculate the percentage for the first top3_hiv_topics_HIV
hiv1_percentage_HIV <- (hiv_sorted_topics_HIV[1] / hiv_total_respondents_HIV) * 100

# Calculate the percentage for the second top3_hiv_topics_HIV
hiv2_percentage_HIV <- (hiv_sorted_topics_HIV[2] / hiv_total_respondents_HIV) * 100

# Calculate the percentage for the third top3_hiv_topics_HIV
hiv3_percentage_HIV <- (hiv_sorted_topics_HIV[3] / hiv_total_respondents_HIV) * 100

# Print the results
hiv1_percentage_HIV
hiv2_percentage_HIV
hiv3_percentage_HIV

Q198 
52.64055

Q199 
37.30835

Q200 
35.43441

## Prep training statistics for HIV providers

In [104]:
# HIV providers - Create a vector of column names representing the PrEP topics columns
prep_topics_HIV <- c("Q211", "Q212", "Q213", "Q214", "Q215","Q216","Q217","Q218","Q219","Q220","Q221","Q222","Q223","Q224","Q225")

### PrEP topic with greatest number of 1st rankings among HIV providers

In [105]:
# HIV providers - Calculate the counts of 1st rankings for each PrEP topic
prep1_count_HIV <- sapply(prep_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 1, na.rm = TRUE))
prep1_count_HIV

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
 124   73   62  108   11   22   20   48   37   31   19   29   36   12    3

In [106]:
# HIV providers - Find the PrEP topic with the highest count of 1st rankings
prep1_rank_HIV <- prep_topics_HIV[which.max(prep1_count_HIV)]
prep1_rank_HIV

[1] "Q211"

In [107]:
# HIV providers - Calculate the percentage of 1st rankings
prep1_perc_HIV <- (prep1_count_HIV[which.max(prep1_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, prep_topics_HIV])) > 0)) * 100
prep1_perc_HIV

Q211 
19.2846

### PrEP topic with greatest number of 2nd rankings among HIV providers

In [108]:
# HIV providers - Calculate the counts of 2nd rankings for each PrEP topic
prep2_count_HIV <- sapply(prep_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 2, na.rm = TRUE))
prep2_count_HIV

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  29   70   53   80   26   34   37   77   50   39   24   27   41   15    0

In [109]:
# HIV providers - Find the PrEP topic with the highest count of 2nd rankings
prep2_rank_HIV <- prep_topics_HIV[which.max(prep2_count_HIV)]
prep2_rank_HIV

[1] "Q214"

In [110]:
# HIV providers - Calculate the percentage of 2nd ranking
prep2_perc_HIV <- (prep2_count_HIV[which.max(prep2_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, prep_topics_HIV])) > 0)) * 100
prep2_perc_HIV

Q214 
12.44168

### PrEP topic with greatest number of 3rd rankings among HIV providers

In [111]:
# HIV providers - Calculate the counts of 3rd rankings for each PrEP topic
prep3_count_HIV <- sapply(prep_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 3, na.rm = TRUE))
prep3_count_HIV

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  44   21   43   52   18   35   38   69   26   66   42   46   42   41    5

In [112]:
# HIV providers - Find the PrEP topic with the highest count of 3rd rankings
prep3_rank_HIV <- prep_topics_HIV[which.max(prep3_count_HIV)]
prep3_rank_HIV

[1] "Q218"

In [113]:
# HIV providers - Calculate the percentage of 3rd rankings
prep3_perc_HIV <- (prep3_count_HIV[which.max(prep3_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, prep_topics_HIV])) > 0)) * 100
prep3_perc_HIV

Q218 
10.73095

### Top 3 PrEP topics among HIV providers

In [114]:
# Calculate the counts for each PrEP topic
prep_topic_counts_HIV <- sapply(prep_topics_HIV, function(topic) sum(HIV_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
prep_sorted_topics_HIV <- sort(prep_topic_counts_HIV, decreasing = TRUE)

# Get the top 3 topics
top3_prep_topics_HIV <- names(prep_sorted_topics_HIV)[1:3]

top3_prep_topics_HIV

[1] "Q214" "Q211" "Q218"

### Percentage of each top 3 PrEP topic among HIV providers

In [115]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
prep_total_respondents_HIV <- sum(rowSums(!is.na(HIV_providers[, prep_topics_HIV])) > 0)

# Calculate the percentage for the first top3_hiv_topics_HIV
prep1_percentage_HIV <- (prep_sorted_topics_HIV[1] / prep_total_respondents_HIV) * 100

# Calculate the percentage for the second top3_hiv_topics_HIV
prep2_percentage_HIV <- (prep_sorted_topics_HIV[2] / prep_total_respondents_HIV) * 100

# Calculate the percentage for the third top3_hiv_topics_HIV
prep3_percentage_HIV <- (prep_sorted_topics_HIV[3] / prep_total_respondents_HIV) * 100

# Print the results
prep1_percentage_HIV
prep2_percentage_HIV
prep3_percentage_HIV

Q214 
37.32504

Q211 
30.63764

Q218 
30.17107

## LGBTQ training statistcis for HIV providers

In [116]:
# Create a vector of column names representing the LGBTQ topics columns
lgbtq_topics_HIV <- c("Q227", "Q228", "Q229", "Q230", "Q231","Q232","Q233","Q234","Q235")

### LGBTQ topic with greatest number of 1st rankings among HIV providers

In [117]:
# HIV providers - Calculate the counts of 1st rankings for each LGBTQ topic
lgbtq1_count_HIV <- sapply(lgbtq_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 1, na.rm = TRUE))
lgbtq1_count_HIV

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
 132  186  113  110   29   63  112  109   11

In [118]:
# HIV providers - Find the LGBTQ topic with the highest count of 1st rankings
lgbtq1_rank_HIV <- lgbtq_topics_HIV[which.max(lgbtq1_count_HIV)]
lgbtq1_rank_HIV

[1] "Q228"

In [119]:
# HIV providers - Calculate the percentage of 1st rankings
lgbtq1_perc_HIV <- (lgbtq1_count_HIV[which.max(lgbtq1_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, lgbtq_topics_HIV])) > 0)) * 100
lgbtq1_perc_HIV

Q228 
21.20867

### LGBTQ topic with greatest number of 2nd rankings among HIV providers

In [120]:
# HIV providers - Calculate the counts of 2nd rankings for each LGBTQ topic
lgbtq2_count_HIV <- sapply(lgbtq_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 2, na.rm = TRUE))
lgbtq2_count_HIV

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  63  114   99  142   58   73  128  134    3

In [121]:
# HIV providers - Find the LGBTQ topic with the highest count of 2nd rankings
lgbtq2_rank_HIV <- lgbtq_topics_HIV[which.max(lgbtq2_count_HIV)]
lgbtq2_rank_HIV

[1] "Q230"

In [122]:
# HIV providers - Calculate the percentage of 2nd rankings
lgbtq2_perc_HIV <- (lgbtq2_count_HIV[which.max(lgbtq2_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, lgbtq_topics_HIV])) > 0)) * 100
lgbtq2_perc_HIV

Q230 
16.19156

### LGBTQ topic with greatest number of 3rd rankings among HIV providers

In [123]:
# HIV providers - Calculate the counts of 3rd rankings for each LGBTQ topic
lgbtq3_count_HIV <- sapply(lgbtq_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 3, na.rm = TRUE))
lgbtq3_count_HIV

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  74  115   69  115   63   60  104  177   13

In [124]:
# HIV providers - Find the LGBTQ topic with the highest count of 3rd rankings
lgbtq3_rank_HIV <- lgbtq_topics_HIV[which.max(lgbtq3_count_HIV)]
lgbtq3_rank_HIV

[1] "Q234"

In [125]:
# HIV providers - Calculate the percentage of 3rd rankings
lgbtq3_perc_HIV <- (lgbtq3_count_HIV[which.max(lgbtq3_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, lgbtq_topics_HIV])) > 0)) * 100
lgbtq3_perc_HIV

Q234 
20.18244

### Top 3 LGBTQ topics among HIV providers

In [126]:
# Calculate the counts for each LGBTQ topic
lgbtq_topic_counts_HIV <- sapply(lgbtq_topics_HIV, function(topic) sum(HIV_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
lgbtq_sorted_topics_HIV <- sort(lgbtq_topic_counts_HIV, decreasing = TRUE)

# Get the top 3 topics
top3_lgbtq_topics_HIV <- names(lgbtq_sorted_topics_HIV)[1:3]
top3_lgbtq_topics_HIV

[1] "Q234" "Q228" "Q230"

### Percentage of each top 3 LGBTQ topic among HIV providers

In [127]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
lgbtq_total_respondents_HIV <- sum(rowSums(!is.na(HIV_providers[, lgbtq_topics_HIV])) > 0)

# Calculate the percentage for the first top3_hiv_topics_HIV
lgbtq1_percentage_HIV <- (lgbtq_sorted_topics_HIV[1] / lgbtq_total_respondents_HIV) * 100

# Calculate the percentage for the second top3_hiv_topics_HIV
lgbtq2_percentage_HIV <- (lgbtq_sorted_topics_HIV[2] / lgbtq_total_respondents_HIV) * 100

# Calculate the percentage for the third top3_hiv_topics_HIV
lgbtq3_percentage_HIV <- (lgbtq_sorted_topics_HIV[3] / lgbtq_total_respondents_HIV) * 100

# Print the results
lgbtq1_percentage_HIV
lgbtq2_percentage_HIV
lgbtq3_percentage_HIV

Q234 
47.89054

Q228 
47.32041

Q230 
41.84721

## Identifying LGBTQ providers

In [128]:
# LGBTQ providers
LGBTQ_providers <- final_data[final_data$Q275 == "Yes" | final_data$Q130 == 1 | final_data$Q1 == "LBGTQ Health Center/Community Center" | final_data$Q22 == "Yes" | final_data$Q33 == 1, ]
head(LGBTQ_providers, 10)

email,Record_ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,⋯,Q810,Q811,Q812,Q813,Q814,Q815,Q816,Q817,Q818,Q819
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
,1136,LBGTQ Health Center/Community Center,NA,Connecticut,6511,Urban,Yes,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
19tyler@gmail.com,422,Non-profit organization,NA,District of Columbia,20005,Urban,No,1,NA,⋯,fear of how far laws will go and stigmatizingp -,Unsure,fear,No,NA,Yes,Yes,gov refuses to use evidenced based data,"limitations on providing harm Reduction services, syringe exchanges and safe injection sites in heavily using areas.",Complete
3sollenberger@gmail.com,1294,Non-profit organization,NA,Pennsylvania,15222,Urban,No,1,1,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
ASHLEY.HART@FLHEALTH.GOV,794,"Government entity (Local, State, Federal)",NA,Florida,32066,Rural,No,1,1,⋯,NA,Unsure,NA,No,NA,No,NA,NA,"WE ARE A VERY SMALL CHD. WE HAVE A SMALL POPULATION AND IF TESTING IS SOUGHT USUALLY PTS WILL GO TO A LARGER CHD WHERE THEY CAN 1. STAY ANONYMOUS, 2 OBTAIN MEDICATIONS/TREATMENT ON SITE 3. SEE A PROVIDER WITH MORE RESOURCES. WE HAVE ONE PROVIDER BETWEEN 2 FACILITIES WHO IS AT OUR LARGER FACILITY 4 DAYS A WEEK AND 2.5 DAYS A MONTH AT OUR SMALL FACILITY. WE ARE CURENTLY SEEKING ANOTHER PROVIDER AS OUR CURRENT ONE IS RETIRING.",Complete
ASIMS@ABSOLUTECARE.COM,1188,Private hospital/clinic setting,NA,Louisiana,70130,Urban,Yes,1,NA,⋯,NA,Unsure,NA,No,NA,No,NA,NA,NA,Complete
Adam.lake@pennmedicine.upenn.edu,901,Academic hospital/clinic setting,NA,Pennsylvania,17602,Suburban,Yes,1,1,⋯,Worse mental health in trans community,Yes,Not sure where to start,No,NA,Yes,Yes,I haven't been able to provide colocated services or openly partner with the harm reduction coalition due to my health system's concern over legality,NA,Complete
Admin@connectdcare.com,123,Private practice,NA,Delaware,19901,Rural,No,1,1,⋯,Yes,Yes,It has,Yes,NA,Yes,Yes,It does,NA,Complete
Aevans@tulane.edu,473,Academic hospital/clinic setting,NA,Louisiana,71301,Suburban,Yes,1,1,⋯,NA,Unsure,NA,No,NA,No,NA,NA,NA,Complete


## Number of incomplete and complete responses among LGBTQ providers

In [129]:
table(LGBTQ_providers$Q819)


  Complete Incomplete 
       754        269 

## Descriptive Statistics for LGBTQ providers

### Age counts among LGBTQ providers

In [130]:
## LGBTQ providers - Age
age_LGBTQ <- table(LGBTQ_providers$Q162)
age_LGBTQ


                18-24                 25-34                 35-44 
                   31                   196                   241 
                45-54                 55-64                    65 
                  243                   202                    88 
Prefer not to respond 
                   19 

### Age percentages among LGBTQ providers

In [131]:
age_perc_LGBTQ <- prop.table(age_LGBTQ)*100
age_perc_LGBTQ


                18-24                 25-34                 35-44 
             3.039216             19.215686             23.627451 
                45-54                 55-64                    65 
            23.823529             19.803922              8.627451 
Prefer not to respond 
             1.862745 

### Gender counts among LGBTQ providers

In [132]:
## LGBTQ providers - Gender
gender_LGBTQ <- table(LGBTQ_providers$Q157)
gender_LGBTQ


                                Female/Woman 
                                         625 
Genderqueer/gender non-confirming/non-binary 
                                          65 
                                    Male/Man 
                                         271 
                       Prefer not to respond 
                                          17 
                     Prefer to self-describe 
                                           9 
                             Transgender man 
                                          19 
                           Transgender woman 
                                          14 

### Gender percentages among LGBTQ providers

In [133]:
gender_perc_LGBTQ <- prop.table(gender_LGBTQ)*100
gender_perc_LGBTQ


                                Female/Woman 
                                  61.2745098 
Genderqueer/gender non-confirming/non-binary 
                                   6.3725490 
                                    Male/Man 
                                  26.5686275 
                       Prefer not to respond 
                                   1.6666667 
                     Prefer to self-describe 
                                   0.8823529 
                             Transgender man 
                                   1.8627451 
                           Transgender woman 
                                   1.3725490 

### Race counts among LGBTQ providers

In [134]:
## LGBTQ providers - Race
race_LGBTQ <- table(LGBTQ_providers$Q159)
race_LGBTQ


        Asian and/or Asian American           Black or African American 
                                 25                                 251 
                        Multiracial    Native American or Alaska Native 
                                 73                                  11 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                                  5                                  53 
            Prefer to self-describe                  White or Caucasian 
                                 24                                 577 

### Race percentages among LGBTQ providers

In [135]:
race_perc_LGBTQ <- prop.table(race_LGBTQ)*100
race_perc_LGBTQ


        Asian and/or Asian American           Black or African American 
                          2.4533857                          24.6319921 
                        Multiracial    Native American or Alaska Native 
                          7.1638862                           1.0794897 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                          0.4906771                           5.2011776 
            Prefer to self-describe                  White or Caucasian 
                          2.3552502                          56.6241413 

### Hispanic/Latinx percentages among LGBTQ providers

In [136]:
## LGBTQ providers - Hispanic/Latinx
hisp_LGBTQ <- table(LGBTQ_providers$Q161)
hisp_LGBTQ


                   No Prefer not to respond                   Yes 
                  814                    19                   187 

### Hispanic/Latinx percentages among LGBTQ providers

In [137]:
hisp_perc_LGBTQ <- prop.table(hisp_LGBTQ)*100
hisp_perc_LGBTQ 


                   No Prefer not to respond                   Yes 
            79.803922              1.862745             18.333333 

### Geographic counts among LGBTQ providers

In [138]:
geog_counts_LGBTQ <- table(LGBTQ_providers$Q5)
geog_counts_LGBTQ


   Rural Suburban   Tribal    Urban 
     206      175        3      639 

### Geographic percentages among LGBTQ providers

In [139]:
geog_perc_LGBTQ <- prop.table(geog_counts_LGBTQ)*100
geog_perc_LGBTQ


     Rural   Suburban     Tribal      Urban 
20.1368524 17.1065494  0.2932551 62.4633431 

### Setting counts among LGBTQ providers

In [140]:
setting_counts_LGBTQ <- table(LGBTQ_providers$Q1)
setting_counts_LGBTQ


             Academic hospital/clinic setting 
                                           74 
                        Advocacy organization 
                                            4 
              AIDS Service Organization (ASO) 
                                          133 
           Community Based Organization (CBO) 
                                          116 
     Federally Qualified Health Center (FQHC) 
                                          134 
    Government entity (Local, State, Federal) 
                                           88 
               Health department (non-clinic) 
                                           38 
Health department clinic/public health clinic 
                                          137 
         LBGTQ Health Center/Community Center 
                                           55 
                      Non-profit organization 
                                          138 
                                        Other 
            

### Setting percentages among LGBTQ providers

In [141]:
setting_perc_LGBTQ <- prop.table(setting_counts_LGBTQ)*100
setting_perc_LGBTQ


             Academic hospital/clinic setting 
                                    7.2336266 
                        Advocacy organization 
                                    0.3910068 
              AIDS Service Organization (ASO) 
                                   13.0009775 
           Community Based Organization (CBO) 
                                   11.3391984 
     Federally Qualified Health Center (FQHC) 
                                   13.0987292 
    Government entity (Local, State, Federal) 
                                    8.6021505 
               Health department (non-clinic) 
                                    3.7145650 
Health department clinic/public health clinic 
                                   13.3919844 
         LBGTQ Health Center/Community Center 
                                    5.3763441 
                      Non-profit organization 
                                   13.4897361 
                                        Other 
            

### Role type counts among LGBTQ providers - clinical vs nonclinical

In [142]:
role_type_counts_LGBTQ <- table(LGBTQ_providers$Q65)
role_type_counts_LGBTQ


    Clinical Non-clinical 
         336          685 

### Role type percentages among LGBTQ providers - clinical vs nonclinical

In [143]:
role_type_perc_LGBTQ <- prop.table(role_type_counts_LGBTQ)*100
role_type_perc_LGBTQ


    Clinical Non-clinical 
    32.90891     67.09109 

### Clinical role counts among LGBTQ providers

In [144]:
role_counts_LGBTQ <- table(LGBTQ_providers$Q66)
role_counts_LGBTQ


                             Dentist      Disease Intervention Specialist 
                                   2                                    1 
     Licensed Clinical Social Worker             Licensed Practical Nurse 
                                  39                                    7 
                   Medical Assistant           Mental Health Case Manager 
                                  10                                    3 
             Mental Health Counselor           Mental Health Professional 
                                   1                                   12 
                       Nurse Manager                        Nurse Midwife 
                                  17                                    1 
                  Nurse Practitioner Other Provider/Clinical professional 
                                  68                                    7 
                          Pharmacist                  Pharmacy Technician 
                        

### Clinical role percentages among LGBTQ providers

In [145]:
role_perc_LGBTQ <- prop.table(role_counts_LGBTQ)*100
role_perc_LGBTQ


                             Dentist      Disease Intervention Specialist 
                           0.5970149                            0.2985075 
     Licensed Clinical Social Worker             Licensed Practical Nurse 
                          11.6417910                            2.0895522 
                   Medical Assistant           Mental Health Case Manager 
                           2.9850746                            0.8955224 
             Mental Health Counselor           Mental Health Professional 
                           0.2985075                            3.5820896 
                       Nurse Manager                        Nurse Midwife 
                           5.0746269                            0.2985075 
                  Nurse Practitioner Other Provider/Clinical professional 
                          20.2985075                            2.0895522 
                          Pharmacist                  Pharmacy Technician 
                        

### Non-clinical role counts among LGBTQ providers

In [146]:
nonclinic_role_counts_LGBTQ <- table(LGBTQ_providers$Q71)
nonclinic_role_counts_LGBTQ


                         Administrator                               Advocate 
                                   160                                     15 
     Case Manager/Medical case manager        Clergy/Faith-Based Professional 
                                   122                                      1 
               Community Health Worker        Disease Intervention Specialist 
                                    63                                     36 
                Finance/Fiscal Manager    Harm Reductionist/Risk Reductionist 
                                     2                                     29 
             Health and Wellness Coach            Health Education Specialist 
                                     1                                     55 
                      Health Navigator                HIV Linkage Coordinator 
                                    35                                      4 
             HIV Retention Coordinator             

### Non-clinical role percentages among LGBTQ providers

In [147]:
nonclinic_role_perc_HIV <- prop.table(nonclinic_role_counts_HIV)*100
nonclinic_role_perc_HIV


                         Administrator                               Advocate 
                            22.9532164                              2.1929825 
     Case Manager/Medical case manager        Clergy/Faith-Based Professional 
                            17.9824561                              0.1461988 
               Community Health Worker        Disease Intervention Specialist 
                             9.3567251                              5.4093567 
                Finance/Fiscal Manager    Harm Reductionist/Risk Reductionist 
                             0.2923977                              4.6783626 
             Health and Wellness Coach            Health Education Specialist 
                             0.1461988                              8.1871345 
                      Health Navigator                HIV Linkage Coordinator 
                             5.1169591                              0.5847953 
             HIV Retention Coordinator             

### Years in role counts among LGBTQ providers

In [148]:
years_counts_LGBTQ <- table(LGBTQ_providers$Q94)
years_counts_LGBTQ


   <2 years 11-20 years   2-4 years   21+ years  5-10 years 
        284         146         216         153         222 

### Years in role percentages among LGBTQ providers

In [149]:
years_perc_LGBTQ <- prop.table(years_counts_LGBTQ)*100
years_perc_LGBTQ


   <2 years 11-20 years   2-4 years   21+ years  5-10 years 
   27.81587    14.29971    21.15573    14.98531    21.74339 

### Prescriber counts among LGBTQ providers

In [150]:
prescriber_counts_LGBTQ <- table(LGBTQ_providers$Q70)
prescriber_counts_LGBTQ


 No Yes 
193 146 

### Prescriber percentages among LGBTQ providers

In [151]:
prescriber_perc_LGBTQ <- prop.table(prescriber_counts_LGBTQ)*100
prescriber_perc_LGBTQ


      No      Yes 
56.93215 43.06785 

## Onsite pharmacy counts among LGBTQ providers

In [152]:
onsite_pharm_counts_LGBTQ <- table(LGBTQ_providers$Q6)
onsite_pharm_counts_LGBTQ


 No Yes 
613 410 

## Onsite pharmacy percentages among LGBTQ providers

In [153]:
onsite_pharm_perc_LGBTQ <- prop.table(onsite_pharm_counts_LGBTQ)*100
onsite_pharm_perc_LGBTQ


     No     Yes 
59.9218 40.0782 

## HIV training statistcis for LGBTQ providers

In [154]:
# LGBTQ providers - Create a vector of column names representing the HIV topics columns
hiv_topics_LGBTQ <- c("Q198", "Q199", "Q200", "Q201", "Q202","Q203","Q204","Q205","Q206","Q207","Q208","Q209","Q210a","Q210b")

### HIV topic with greatest number of 1st rankings among LGBTQ providers

In [155]:
# LGBTQ providers - Calculate the counts of 1st rankings for each HIV topic
hiv1_count_LGBTQ <- sapply(hiv_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 1, na.rm = TRUE))
hiv1_count_LGBTQ

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  178    80    72    17    21    37    34    49    48     6    12    12     3 
Q210b 
    1

In [156]:
# LGBTQ providers - Find the HIV topic with the highest count of 1st rankings
hiv1_rank_LGBTQ <- hiv_topics_LGBTQ[which.max(hiv1_count_LGBTQ)]
hiv1_rank_LGBTQ

[1] "Q198"

In [157]:
# LGBTQ providers - Calculate the percentage of 1st rankings
hiv1_perc_LGBTQ <- (hiv1_count_LGBTQ[which.max(hiv1_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, hiv_topics_LGBTQ])) > 0)) * 100
hiv1_perc_LGBTQ

Q198 
30.95652

### HIV topic with greatest number of 2nd rankings among LGBTQ providers

In [158]:
# LGBTQ providers - Calculate the counts of 2nd rankings for each HIV topic
hiv2_count_LGBTQ <- sapply(hiv_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 2, na.rm = TRUE))
hiv2_count_LGBTQ

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   56    87    76    26    42    51    48    49    56    15    19     2     0 
Q210b 
    1

In [159]:
# LGBTQ providers - Find the HIV topic with the highest count of 2nd rankings
hiv2_rank_LGBTQ <- hiv_topics_LGBTQ[which.max(hiv2_count_LGBTQ)]
hiv2_rank_LGBTQ

[1] "Q199"

In [160]:
# LGBTQ providers - Calculate the percentage of 2nd rankings
hiv2_perc_LGBTQ  <- (hiv2_count_LGBTQ[which.max(hiv2_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, hiv_topics_LGBTQ])) > 0)) * 100
hiv2_perc_LGBTQ 

Q199 
15.13043

### HIV topic with greatest number of 3rd rankings among LGBTQ providers

In [161]:
# LGBTQ providers - Calculate the counts of 3rd rankings for each HIV topic
hiv3_count_LGBTQ  <- sapply(hiv_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 3, na.rm = TRUE))
hiv3_count_LGBTQ

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   65    47    53    17    56    46    57    58    52    12    36    16     2 
Q210b 
    3

In [162]:
# LGBTQ providers - Find the PrEP topic with the highest count of 3rd rankings
hiv3_rank_LGBTQ <- hiv_topics_LGBTQ[which.max(hiv3_count_LGBTQ)]
hiv3_rank_LGBTQ

[1] "Q198"

In [163]:
# LGBTQ providers - Calculate the percentage of 3rd ranking
hiv3_perc_LGBTQ <- (hiv3_count_LGBTQ[which.max(hiv3_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, hiv_topics_LGBTQ])) > 0)) * 100
hiv3_perc_LGBTQ

Q198 
11.30435

### Top 3 HIV topics among LGBTQ providers

In [164]:
# Calculate the counts for each HIV topic
hiv_topic_counts_LGBTQ <- sapply(hiv_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))
hiv_topic_counts_LGBTQ
# Sort the topics based on their counts in descending order
hiv_sorted_topics_LGBTQ <- sort(hiv_topic_counts_LGBTQ, decreasing = TRUE)
hiv_sorted_topics_LGBTQ
# Get the top 3 topics
top3_hiv_topics_LGBTQ <- names(hiv_sorted_topics_LGBTQ)[1:3]

top3_hiv_topics_LGBTQ

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  299   214   201    60   119   134   139   156   156    33    67    30     5 
Q210b 
    5

Q198  Q199  Q200  Q205  Q206  Q204  Q203  Q202  Q208  Q201  Q207  Q209 Q210a 
  299   214   201   156   156   139   134   119    67    60    33    30     5 
Q210b 
    5

[1] "Q198" "Q199" "Q200"

### Percentage of each top 3 HIV topic among LGBTQ providers

In [165]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
hiv_total_respondents_LGBTQ <- sum(rowSums(!is.na(LGBTQ_providers[, hiv_topics_LGBTQ])) > 0)

# Calculate the percentage for the first top3_hiv_topics_LGBTQ
hiv1_percentage_LGBTQ <- (hiv_sorted_topics_LGBTQ[1] / hiv_total_respondents_LGBTQ) * 100

# Calculate the percentage for the second top3_hiv_topics_LGBTQ
hiv2_percentage_LGBTQ <- (hiv_sorted_topics_LGBTQ[2] / hiv_total_respondents_LGBTQ) * 100

# Calculate the percentage for the third top3_hiv_topics_LGBTQ
hiv3_percentage_LGBTQ <- (hiv_sorted_topics_LGBTQ[3] / hiv_total_respondents_LGBTQ) * 100

# Print the results
hiv1_percentage_LGBTQ
hiv2_percentage_LGBTQ
hiv3_percentage_LGBTQ

Q198 
  52

Q199 
37.21739

Q200 
34.95652

## PrEP training statistcis for LGBTQ providers

In [166]:
# LGBTQ providers - Create a vector of column names representing the PrEP topics columns
prep_topics_LGBTQ <- c("Q211", "Q212", "Q213", "Q214", "Q215","Q216","Q217","Q218","Q219","Q220","Q221","Q222","Q223","Q224","Q225")

### PrEP topic with greatest number of 1st rankings among LGBTQ providers

In [167]:
# LGBTQ providers - Calculate the counts of 1st rankings for each PrEP topic
prep1_count_LGBTQ <- sapply(prep_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 1, na.rm = TRUE))
prep1_count_LGBTQ

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
 124   73   62  108   11   22   20   48   37   31   19   29   36   12    3

In [168]:
# LGBTQ providers - Find the PrEP topic with the highest count of 1st rankings
prep1_rank_LGBTQ <- prep_topics_LGBTQ[which.max(prep1_count_LGBTQ)]
prep1_rank_LGBTQ

[1] "Q211"

In [169]:
# LGBTQ providers - Calculate the percentage of 1st rankings
prep1_perc_LGBTQ <- (prep1_count_LGBTQ[which.max(prep1_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, prep_topics_LGBTQ])) > 0)) * 100
prep1_perc_LGBTQ

Q211 
19.2846

### PrEP topic with greatest number of 2nd rankings among LGBTQ providers

In [170]:
# LGBTQ providers - Calculate the counts of 2nd rankings for each PrEP topic
prep2_count_LGBTQ <- sapply(prep_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 2, na.rm = TRUE))
prep2_count_LGBTQ

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  29   70   53   80   26   34   37   77   50   39   24   27   41   15    0

In [171]:
# LGBTQ providers - Find the PrEP topic with the highest count of 2nd rankings
prep2_rank_LGBTQ <- prep_topics_LGBTQ[which.max(prep2_count_LGBTQ)]
prep2_rank_LGBTQ

[1] "Q214"

In [172]:
# LGBTQ providers - Calculate the percentage of 2nd rankings
prep2_perc_LGBTQ  <- (prep2_count_LGBTQ[which.max(prep2_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, prep_topics_LGBTQ])) > 0)) * 100
prep2_perc_LGBTQ 

Q214 
12.44168

### PrEP topic with greatest number of 3rd rankings among LGBTQ providers

In [173]:
# LGBTQ providers - Calculate the counts of 3rd rankings for each PrEP topic
prep3_count_LGBTQ  <- sapply(prep_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 3, na.rm = TRUE))
prep3_count_LGBTQ

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  44   21   43   52   18   35   38   69   26   66   42   46   42   41    5

In [174]:
# LGBTQ providers - Find the PrEP topic with the highest count of 3rd rankings
prep3_rank_LGBTQ <- prep_topics_LGBTQ[which.max(prep3_count_LGBTQ )]
prep3_rank_LGBTQ

[1] "Q218"

In [175]:
# LGBTQ providers - Calculate the percentage of 3rd ranking
prep3_perc_LGBTQ <- (prep3_count_LGBTQ[which.max(prep3_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, prep_topics_LGBTQ])) > 0)) * 100
prep3_perc_LGBTQ

Q218 
10.73095

### Top 3 PrEP topics among LGBTQ providers

In [176]:
# Calculate the counts for each PrEP topic
prep_topic_counts_LGBTQ <- sapply(prep_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
prep_sorted_topics_LGBTQ <- sort(prep_topic_counts_LGBTQ, decreasing = TRUE)

# Get the top 3 topics
top3_prep_topics_LGBTQ <- names(prep_sorted_topics_LGBTQ)[1:3]
top3_prep_topics_LGBTQ

[1] "Q214" "Q211" "Q218"

### Percentage of each top 3 PrEP topic among LGBTQ providers

In [177]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the PrEP topic questions
prep_total_respondents_LGBTQ <- sum(rowSums(!is.na(LGBTQ_providers[, prep_topics_LGBTQ])) > 0)

# Calculate the percentage for the first top3_prep_topics_LGBTQ
prep1_percentage_LGBTQ <- (prep_sorted_topics_LGBTQ[1] / prep_total_respondents_LGBTQ) * 100

# Calculate the percentage for the second top3_prep_topics_LGBTQ
prep2_percentage_LGBTQ <- (prep_sorted_topics_LGBTQ[2] / prep_total_respondents_LGBTQ) * 100

# Calculate the percentage for the third top3_prep_topics_LGBTQ
prep3_percentage_LGBTQ <- (prep_sorted_topics_LGBTQ[3] / prep_total_respondents_LGBTQ) * 100

# Print the results
prep1_percentage_LGBTQ
prep2_percentage_LGBTQ
prep3_percentage_LGBTQ

Q214 
37.32504

Q211 
30.63764

Q218 
30.17107

## LGBTQ training statistcis for LGBTQ providers

In [178]:
# Create a vector of column names representing the LGBTQ topics columns
lgbtq_topics_LGBTQ <- c("Q227", "Q228", "Q229", "Q230", "Q231","Q232","Q233","Q234","Q235")

### LGBTQ topic with greatest number of 1st rankings among LGBTQ providers

In [179]:
# LGBTQ providers - Calculate the counts of 1st rankings for each LGBTQ topic
lgbtq1_count_LGBTQ <- sapply(lgbtq_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 1, na.rm = TRUE))
lgbtq1_count_LGBTQ

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
 132  178  114  111   29   58  110  108   14

In [180]:
# LGBTQ providers - Find the LGBTQ topic with the highest count of 1st rankings
lgbtq1_rank_LGBTQ <- lgbtq_topics_LGBTQ[which.max(lgbtq1_count_LGBTQ)]
lgbtq1_rank_LGBTQ

[1] "Q228"

In [181]:
# LGBTQ providers - Calculate the percentage of 1st rankings
lgbtq1_perc_LGBTQ <- (lgbtq1_count_LGBTQ[which.max(lgbtq1_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, lgbtq_topics_LGBTQ])) > 0)) * 100
lgbtq1_perc_LGBTQ

Q228 
20.57803

### LGBTQ topic with greatest number of 2nd rankings among LGBTQ providers

In [182]:
# LGBTQ providers - Calculate the counts of 2nd rankings for each LGBTQ topic
lgbtq2_count_LGBTQ <- sapply(lgbtq_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 2, na.rm = TRUE))
lgbtq2_count_LGBTQ

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  62  113  101  137   60   71  128  128    3

In [183]:
# LGBTQ providers - Find the LGBTQ topic with the highest count of 2nd rankings
lgbtq2_rank_LGBTQ <- lgbtq_topics_LGBTQ[which.max(lgbtq2_count_LGBTQ)]
lgbtq2_rank_LGBTQ

[1] "Q230"

In [184]:
# LGBTQ providers - Calculate the percentage of 2nd rankings
lgbtq2_perc_LGBTQ <- (lgbtq2_count_LGBTQ[which.max(lgbtq2_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, lgbtq_topics_LGBTQ])) > 0)) * 100
lgbtq2_perc_LGBTQ

Q230 
15.83815

### LGBTQ topic with greatest number of 3rd rankings among LGBTQ providers

In [185]:
# LGBTQ providers - Calculate the counts of 3rd rankings for each LGBTQ topic
lgbtq3_count_LGBTQ <- sapply(lgbtq_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 2, na.rm = TRUE))
lgbtq3_count_LGBTQ

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  62  113  101  137   60   71  128  128    3

In [186]:
# LGBTQ providers - Find the LGBTQ topic with the highest count of 2nd rankings
lgbtq3_rank_LGBTQ <- lgbtq_topics_LGBTQ[which.max(lgbtq3_count_LGBTQ)]
lgbtq3_rank_LGBTQ

[1] "Q230"

In [187]:
# LGBTQ providers - Calculate the percentage of 2nd rankings
lgbtq3_perc_LGBTQ <- (lgbtq3_count_LGBTQ[which.max(lgbtq3_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, lgbtq_topics_LGBTQ])) > 0)) * 100
lgbtq3_perc_LGBTQ

Q230 
15.83815

### Top 3 LGBTQ topics among LGBTQ providers

In [188]:
# Calculate the counts for each LGBTQ topic
lgbtq_topic_counts_LGBTQ <- sapply(lgbtq_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
lgbtq_sorted_topics_LGBTQ <- sort(lgbtq_topic_counts_LGBTQ, decreasing = TRUE)

# Get the top 3 topics
top3_lgbtq_topics_LGBTQ <- names(lgbtq_sorted_topics_LGBTQ)[1:3]
top3_lgbtq_topics_LGBTQ

[1] "Q234" "Q228" "Q230"

### Percentage of each top 3 LGBTQ topic among LGBTQ providers


In [189]:
# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the LGBTQ topic questions
lgtbq_total_respondents_LGBTQ <- sum(rowSums(!is.na(LGBTQ_providers[, lgbtq_topics_LGBTQ])) > 0)

# Calculate the percentage for the first top3_lgbtq_topics_LGBTQ
lgbtq1_percentage_LGBTQ <- (lgbtq_sorted_topics_LGBTQ[1] / lgtbq_total_respondents_LGBTQ) * 100

# Calculate the percentage for the second top3_lgbtq_topics_LGBTQ
lgbtq2_percentage_LGBTQ <- (lgbtq_sorted_topics_LGBTQ[2] / lgtbq_total_respondents_LGBTQ) * 100

# Calculate the percentage for the third top3_lgbtq_topics_LGBTQ
lgbtq3_percentage_LGBTQ <- (lgbtq_sorted_topics_LGBTQ[3] / lgtbq_total_respondents_LGBTQ) * 100

# Print the results
lgbtq1_percentage_LGBTQ
lgbtq2_percentage_LGBTQ
lgbtq3_percentage_LGBTQ

Q234 
47.74566

Q228 
46.47399

Q230 
42.31214

# Insurance data

In [190]:
# Define the insurance types and their corresponding column names
insurance_counts <- final_data %>%
  summarise(
    Medicaid = sum(Q7 == 1, na.rm = TRUE),
    Medicare = sum(Q8 == 1, na.rm = TRUE),
    Other_Public_Insurance = sum(Q9 == 1, na.rm = TRUE),
    Employer_Sponsored = sum(Q10 == 1, na.rm = TRUE),
    Other_Private = sum(Q11 == 1, na.rm = TRUE),
    Income_Dependent = sum(Q12 == 1, na.rm = TRUE),
    Other_Option = sum(Q13 == 1, na.rm = TRUE)
  )

insurance_counts

Medicaid,Medicare,Other_Public_Insurance,Employer_Sponsored,Other_Private,Income_Dependent,Other_Option
<int>,<int>,<int>,<int>,<int>,<int>,<int>
825,731,663,654,682,606,297


In [191]:
# Calculate the total count across all rows for each insurance option
insurance_columns <- final_data[, c("Q7", "Q8", "Q9", "Q10", "Q11", "Q12", "Q13")]

# Calculate the total count across all types of insurance
total_all_insurance <- sum(insurance_columns, na.rm = TRUE)
total_all_insurance

[1] 4458

In [192]:
# Calculate the percentage of each option
insurance_percentage <- insurance_counts / total_all_insurance * 100

# Print the percentages
insurance_percentage

Medicaid,Medicare,Other_Public_Insurance,Employer_Sponsored,Other_Private,Income_Dependent,Other_Option
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
18.50606,16.39749,14.87214,14.67026,15.29834,13.59354,6.66218


## Insurance data by state

In [193]:
# Calculate the count of each insurance option by state
ins_counts_by_state <- final_data %>%
  group_by(Q3) %>%
  summarise(
    Medicaid = sum(Q7 == 1, na.rm = TRUE),
    Medicare = sum(Q8 == 1, na.rm = TRUE),
    Other_Public_Insurance = sum(Q9 == 1, na.rm = TRUE),
    Employer_Sponsored = sum(Q10 == 1, na.rm = TRUE),
    Other_Private = sum(Q11 == 1, na.rm = TRUE),
    Income_Dependent = sum(Q12 == 1, na.rm = TRUE),
    Other_Option = sum(Q13 == 1, na.rm = TRUE),
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the counts of each insurance option by state
ins_counts_by_state

Q3,Medicaid,Medicare,Other_Public_Insurance,Employer_Sponsored,Other_Private,Income_Dependent,Other_Option,Total_Count
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
Alabama,6,5,5,4,6,4,1,31
Alaska,0,0,0,0,0,0,1,1
Arizona,13,10,7,7,10,9,5,61
Arkansas,7,7,7,6,6,5,2,40
California,73,73,53,48,51,49,32,379
Colorado,12,10,10,9,10,9,5,65
Connecticut,12,12,10,8,7,10,6,65
Delaware,11,11,8,10,10,6,2,58
District of Columbia,26,20,17,14,18,11,9,115


In [194]:
# Calculate the total count across all types of insurance for each state
total_ins_counts_by_state <- final_data %>%
  group_by(Q3) %>%
  summarise(
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the total counts across all types of insurance for each state
total_ins_counts_by_state

Q3,Total_Count
<chr>,<dbl>
Alabama,31
Alaska,1
Arizona,61
Arkansas,40
California,379
Colorado,65
Connecticut,65
Delaware,58
District of Columbia,115


In [195]:
# Calculate the percentage of each option by state
ins_perc_by_state <- ins_counts_by_state %>%
    group_by(Q3) %>%
    summarize(
    Medicaid_Percentage = (Medicaid / Total_Count) * 100,
    Medicare_Percentage = (Medicare / Total_Count) * 100,
    Other_Public_Insurance_Percentage = (Other_Public_Insurance / Total_Count) * 100,
    Employer_Sponsored_Percentage = (Employer_Sponsored / Total_Count) * 100,
    Other_Private_Percentage = (Other_Private / Total_Count) * 100,
    Income_Dependent_Percentage = (Income_Dependent / Total_Count) * 100,
    Other_Option_Percentage = (Other_Option / Total_Count) * 100
)
# Print the percentage of each option by state
ins_perc_by_state

Q3,Medicaid_Percentage,Medicare_Percentage,Other_Public_Insurance_Percentage,Employer_Sponsored_Percentage,Other_Private_Percentage,Income_Dependent_Percentage,Other_Option_Percentage
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Alabama,19.35484,16.12903,16.12903,12.903226,19.354839,12.903226,3.225806
Alaska,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,100.000000
Arizona,21.31148,16.39344,11.47541,11.475410,16.393443,14.754098,8.196721
Arkansas,17.50000,17.50000,17.50000,15.000000,15.000000,12.500000,5.000000
California,19.26121,19.26121,13.98417,12.664908,13.456464,12.928760,8.443272
Colorado,18.46154,15.38462,15.38462,13.846154,15.384615,13.846154,7.692308
Connecticut,18.46154,18.46154,15.38462,12.307692,10.769231,15.384615,9.230769
Delaware,18.96552,18.96552,13.79310,17.241379,17.241379,10.344828,3.448276
District of Columbia,22.60870,17.39130,14.78261,12.173913,15.652174,9.565217,7.826087


## Insurance data by CDC region

In [196]:
# Create a new column "Region" based on the states
final_data <- final_data %>%
  mutate(Region = case_when(
    Q3 %in% c("Connecticut", "Maine", "Massachusetts", "New Hampshire", "New Jersey", "New York", "Pennsylvania", "Rhode Island", "Vermont") ~ "Northeast",
    Q3 %in% c("Illinois", "Indiana", "Iowa", "Kansas", "Michigan", "Minnesota", "Missouri", "Nebraska", "North Dakota", "Ohio", "South Dakota", "Wisconsin") ~ "Midwest",
    Q3 %in% c("Alabama", "Arkansas", "Delaware", "District of Columbia", "Florida", "Georgia", "Kentucky", "Louisiana", "Maryland", "Mississippi", "North Carolina", "Oklahoma", "South Carolina", "Tennessee", "Texas", "Virginia", "West Virginia") ~ "South",
    Q3 %in% c("Alaska", "Arizona", "California", "Colorado", "Hawaii", "Idaho", "Montana", "Nevada", "New Mexico", "Oregon", "Utah", "Washington", "Wyoming") ~ "West",
    Q3 %in% c("Puerto Rico", "Virgin Islands") ~ "Territories",
    TRUE ~ "Unknown" # If the state is NA
  ))

In [197]:
# Filter rows where the Region is "Unknown"
unknown_states <- filter(final_data, Region == "Unknown")

# Print the states categorized as "Unknown"
print(unknown_states$Q3)

[1] NA NA NA NA


In [198]:
# Calculate the count of each insurance option by region
ins_counts_by_region <- final_data %>%
  group_by(Region) %>%
  summarise(
    Medicaid = sum(Q7 == 1, na.rm = TRUE),
    Medicare = sum(Q8 == 1, na.rm = TRUE),
    Other_Public_Insurance = sum(Q9 == 1, na.rm = TRUE),
    Employer_Sponsored = sum(Q10 == 1, na.rm = TRUE),
    Other_Private = sum(Q11 == 1, na.rm = TRUE),
    Income_Dependent = sum(Q12 == 1, na.rm = TRUE),
    Other_Option = sum(Q13 == 1, na.rm = TRUE),
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the counts of each insurance option by state and region
ins_counts_by_region

Region,Medicaid,Medicare,Other_Public_Insurance,Employer_Sponsored,Other_Private,Income_Dependent,Other_Option,Total_Count
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
Midwest,174,151,137,152,154,133,44,945
Northeast,176,155,146,141,144,119,65,946
South,329,287,264,253,268,252,113,1766
Territories,7,8,7,7,6,4,2,41
Unknown,1,1,1,1,1,1,1,7
West,138,129,108,100,109,97,72,753


In [199]:
# Calculate the total count across all types of insurance for each region
total_ins_counts_by_region <- final_data %>%
  group_by(Region) %>%
  summarise(
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the total counts across all types of insurance for each region
total_ins_counts_by_region

Region,Total_Count
<chr>,<dbl>
Midwest,945
Northeast,946
South,1766
Territories,41
Unknown,7
West,753


In [200]:
# Calculate the percentage of each option by region
ins_perc_by_region <- ins_counts_by_region %>%
    group_by(Region) %>%
    summarize(
    Medicaid_Percentage = (Medicaid / Total_Count) * 100,
    Medicare_Percentage = (Medicare / Total_Count) * 100,
    Other_Public_Insurance_Percentage = (Other_Public_Insurance / Total_Count) * 100,
    Employer_Sponsored_Percentage = (Employer_Sponsored / Total_Count) * 100,
    Other_Private_Percentage = (Other_Private / Total_Count) * 100,
    Income_Dependent_Percentage = (Income_Dependent / Total_Count) * 100,
    Other_Option_Percentage = (Other_Option / Total_Count) * 100
  )
# Print the percentage of each option by state and region
ins_perc_by_region

Region,Medicaid_Percentage,Medicare_Percentage,Other_Public_Insurance_Percentage,Employer_Sponsored_Percentage,Other_Private_Percentage,Income_Dependent_Percentage,Other_Option_Percentage
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Midwest,18.41270,15.97884,14.49735,16.08466,16.29630,14.074074,4.656085
Northeast,18.60465,16.38478,15.43340,14.90486,15.22199,12.579281,6.871036
South,18.62967,16.25142,14.94904,14.32616,15.17554,14.269536,6.398641
Territories,17.07317,19.51220,17.07317,17.07317,14.63415,9.756098,4.878049
Unknown,14.28571,14.28571,14.28571,14.28571,14.28571,14.285714,14.285714
West,18.32669,17.13147,14.34263,13.28021,14.47543,12.881806,9.561753


## Insurance data by Medicaid Expansion states

In [201]:
# Create a new column "Medicaid_Expansion_Status" based on Medicaid expansion status
final_data <- final_data  %>%
  mutate(Medicaid_Expansion = case_when(
      Q3 %in% c("Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "District of Columbia", "Hawaii", "Illinois", "Idaho", "Indiana", "Iowa", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "Utah", "Vermont", "Virginia", "Washington", "West Virginia") ~ "Expanded",
      Q3 %in% c("Wyoming", "Kansas", "Texas", "Wisconsin", "Tennessee", "Mississippi", "Alabama", "Georgia", "South Carolina", "Florida") ~ "Not Expanded",
      Q3 %in% c("Puerto Rico", "Virgin Islands") ~ "Territories",
      TRUE ~ "Unknown" # For NA states
  ))

In [202]:
# Filter rows where the Region is "Unknown"
unknown_states <- filter(final_data, Medicaid_Expansion == "Unknown")

# Print the states categorized as "Unknown"
print(unknown_states$Q3)

[1] NA NA NA NA


In [203]:
# Calculate the count of each insurance option by expansion status
ins_counts_by_expansion_status<- final_data %>%
  group_by(Medicaid_Expansion) %>%
  summarise(
    Medicaid = sum(Q7 == 1, na.rm = TRUE),
    Medicare = sum(Q8 == 1, na.rm = TRUE),
    Other_Public_Insurance = sum(Q9 == 1, na.rm = TRUE),
    Employer_Sponsored = sum(Q10 == 1, na.rm = TRUE),
    Other_Private = sum(Q11 == 1, na.rm = TRUE),
    Income_Dependent = sum(Q12 == 1, na.rm = TRUE),
    Other_Option = sum(Q13 == 1, na.rm = TRUE),
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the counts of each insurance option by expansion status
ins_counts_by_expansion_status

Medicaid_Expansion,Medicaid,Medicare,Other_Public_Insurance,Employer_Sponsored,Other_Private,Income_Dependent,Other_Option,Total_Count
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
Expanded,665,589,531,529,541,474,232,3561
Not Expanded,152,133,124,117,134,127,62,849
Territories,7,8,7,7,6,4,2,41
Unknown,1,1,1,1,1,1,1,7


In [204]:
# Calculate the total count across all types of insurance for each expansion status
total_ins_counts_by_expansion_status <- final_data %>%
  group_by(Medicaid_Expansion) %>%
  summarise(
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the total counts across all types of insurance for each expansion status
total_ins_counts_by_expansion_status

Medicaid_Expansion,Total_Count
<chr>,<dbl>
Expanded,3561
Not Expanded,849
Territories,41
Unknown,7


In [205]:
# Calculate the percentage of each option by expansion status
ins_perc_by_expansion_status <- ins_counts_by_expansion_status %>%
    group_by(Medicaid_Expansion) %>%
    summarize(
    Medicaid_Percentage = (Medicaid / Total_Count) * 100,
    Medicare_Percentage = (Medicare / Total_Count) * 100,
    Other_Public_Insurance_Percentage = (Other_Public_Insurance / Total_Count) * 100,
    Employer_Sponsored_Percentage = (Employer_Sponsored / Total_Count) * 100,
    Other_Private_Percentage = (Other_Private / Total_Count) * 100,
    Income_Dependent_Percentage = (Income_Dependent / Total_Count) * 100,
    Other_Option_Percentage = (Other_Option / Total_Count) * 100
  )
# Print the percentage of each option by state and region
ins_perc_by_expansion_status

Medicaid_Expansion,Medicaid_Percentage,Medicare_Percentage,Other_Public_Insurance_Percentage,Employer_Sponsored_Percentage,Other_Private_Percentage,Income_Dependent_Percentage,Other_Option_Percentage
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Expanded,18.67453,16.54030,14.91154,14.85538,15.19236,13.310868,6.515024
Not Expanded,17.90342,15.66549,14.60542,13.78092,15.78327,14.958775,7.302709
Territories,17.07317,19.51220,17.07317,17.07317,14.63415,9.756098,4.878049
Unknown,14.28571,14.28571,14.28571,14.28571,14.28571,14.285714,14.285714


## Insurance data by site

In [206]:
# Calculate the count of each insurance option by site
ins_counts_by_site <- final_data %>%
  group_by(Q1) %>%
  summarise(
    Medicaid = sum(Q7 == 1, na.rm = TRUE),
    Medicare = sum(Q8 == 1, na.rm = TRUE),
    Other_Public_Insurance = sum(Q9 == 1, na.rm = TRUE),
    Employer_Sponsored = sum(Q10 == 1, na.rm = TRUE),
    Other_Private = sum(Q11 == 1, na.rm = TRUE),
    Income_Dependent = sum(Q12 == 1, na.rm = TRUE),
    Other_Option = sum(Q13 == 1, na.rm = TRUE),
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the counts of each insurance option by site
ins_counts_by_site

Q1,Medicaid,Medicare,Other_Public_Insurance,Employer_Sponsored,Other_Private,Income_Dependent,Other_Option,Total_Count
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
AIDS Service Organization (ASO),102,96,82,84,87,69,40,560
Academic hospital/clinic setting,76,68,66,71,74,45,8,408
Advocacy organization,1,0,1,1,1,0,3,7
Community Based Organization (CBO),77,64,52,54,56,51,52,406
Federally Qualified Health Center (FQHC),133,130,117,109,124,122,19,754
"Government entity (Local, State, Federal)",66,57,54,50,47,48,29,351
Health department (non-clinic),24,20,20,16,16,15,16,127
Health department clinic/public health clinic,120,106,93,83,86,101,23,612
LBGTQ Health Center/Community Center,32,27,25,22,25,21,16,168


In [207]:
# Calculate the total count across all types of insurance for each site
total_ins_counts_by_site <- final_data %>%
  group_by(Q1) %>%
  summarise(
    Total_Count = sum(Q7, Q8, Q9, Q10, Q11, Q12, Q13, na.rm = TRUE)
  )
# Print the total counts across all types of insurance for each site
total_ins_counts_by_site

Q1,Total_Count
<chr>,<dbl>
AIDS Service Organization (ASO),560
Academic hospital/clinic setting,408
Advocacy organization,7
Community Based Organization (CBO),406
Federally Qualified Health Center (FQHC),754
"Government entity (Local, State, Federal)",351
Health department (non-clinic),127
Health department clinic/public health clinic,612
LBGTQ Health Center/Community Center,168


In [208]:
# Calculate the percentage of each option by site
ins_perc_by_site <- ins_counts_by_site %>%
    group_by(Q1) %>%
    summarize(
    Medicaid_Percentage = (Medicaid / Total_Count) * 100,
    Medicare_Percentage = (Medicare / Total_Count) * 100,
    Other_Public_Insurance_Percentage = (Other_Public_Insurance / Total_Count) * 100,
    Employer_Sponsored_Percentage = (Employer_Sponsored / Total_Count) * 100,
    Other_Private_Percentage = (Other_Private / Total_Count) * 100,
    Income_Dependent_Percentage = (Income_Dependent / Total_Count) * 100,
    Other_Option_Percentage = (Other_Option / Total_Count) * 100
)
# Print the percentage of each option by site
ins_perc_by_site

Q1,Medicaid_Percentage,Medicare_Percentage,Other_Public_Insurance_Percentage,Employer_Sponsored_Percentage,Other_Private_Percentage,Income_Dependent_Percentage,Other_Option_Percentage
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AIDS Service Organization (ASO),18.21429,17.14286,14.64286,15.00000,15.53571,12.32143,7.142857
Academic hospital/clinic setting,18.62745,16.66667,16.17647,17.40196,18.13725,11.02941,1.960784
Advocacy organization,14.28571,0.00000,14.28571,14.28571,14.28571,0.00000,42.857143
Community Based Organization (CBO),18.96552,15.76355,12.80788,13.30049,13.79310,12.56158,12.807882
Federally Qualified Health Center (FQHC),17.63926,17.24138,15.51724,14.45623,16.44562,16.18037,2.519894
"Government entity (Local, State, Federal)",18.80342,16.23932,15.38462,14.24501,13.39031,13.67521,8.262108
Health department (non-clinic),18.89764,15.74803,15.74803,12.59843,12.59843,11.81102,12.598425
Health department clinic/public health clinic,19.60784,17.32026,15.19608,13.56209,14.05229,16.50327,3.758170
LBGTQ Health Center/Community Center,19.04762,16.07143,14.88095,13.09524,14.88095,12.50000,9.523810


# MPOX Data

## Offering MPOX vax counts

In [209]:
mpox_vax_counts <- table(final_data$Q261)
mpox_vax_counts


 No Yes 
405 565 

## Offering MPOX vax perc

In [210]:
mpox_vax_perc <- prop.table(mpox_vax_counts)*100
mpox_vax_perc


      No      Yes 
41.75258 58.24742 

## MPOX vax offering by organization type counts

In [211]:
# Calculate the count of "Yes" and "No" responses by site for column "Q261"
mpox_vax_counts_by_site <- final_data %>%
  group_by(Q1) %>%
  summarize(
    Yes = sum(Q261 == "Yes", na.rm = TRUE),
    No = sum(Q261 == "No", na.rm = TRUE)
  )

# Print the counts of "Yes," "No," and "Unsure" responses by site for column "Q261"
mpox_vax_counts_by_site

Q1,Yes,No
<chr>,<int>,<int>
AIDS Service Organization (ASO),64,51
Academic hospital/clinic setting,48,24
Advocacy organization,0,3
Community Based Organization (CBO),45,70
Federally Qualified Health Center (FQHC),77,46
"Government entity (Local, State, Federal)",68,22
Health department (non-clinic),31,6
Health department clinic/public health clinic,110,19
LBGTQ Health Center/Community Center,26,18


## MPOX vax offering by organization type percentages

In [212]:
# Calculate the percentage of "Yes" and "No" responses for each site (column "Q1")
mpox_vax_perc_by_site <- final_data %>%
  group_by(Q1) %>%
  summarize(
    Yes_Percentage = sum(Q261 == "Yes", na.rm = TRUE) / n() * 100,
    No_Percentage = sum(Q261 == "No", na.rm = TRUE) / n() * 100
  )

# Print the percentage of "Yes" and "No" responses for each site
mpox_vax_perc_by_site

Q1,Yes_Percentage,No_Percentage
<chr>,<dbl>,<dbl>
AIDS Service Organization (ASO),48.12030,38.34586
Academic hospital/clinic setting,60.00000,30.00000
Advocacy organization,0.00000,60.00000
Community Based Organization (CBO),34.88372,54.26357
Federally Qualified Health Center (FQHC),57.46269,34.32836
"Government entity (Local, State, Federal)",64.76190,20.95238
Health department (non-clinic),75.60976,14.63415
Health department clinic/public health clinic,75.34247,13.01370
LBGTQ Health Center/Community Center,47.27273,32.72727


## High MPOX vaccine uptake among LGBTQ clients/clients/patients counts

In [213]:
mpox_uptake_counts <- table(final_data$Q262)
mpox_uptake_counts


    No Unsure    Yes 
   109    157    300 

## High MPOX vaccine uptake among LGBTQ clients/clients/patients percentages

In [214]:
mpox_uptake_perc <- prop.table(mpox_uptake_counts)*100
mpox_uptake_perc


      No   Unsure      Yes 
19.25795 27.73852 53.00353 

## High MPOX vaccine uptake among LGBTQ clients/clients/patients by organization type counts

In [215]:
# Calculate the count of "Yes", "No", and "Unsure" responses by site for column "Q262"
mpox_uptake_counts_by_site <- final_data %>%
  group_by(Q1) %>%
  summarize(
    Yes = sum(Q262 == "Yes", na.rm = TRUE),
    No = sum(Q262 == "No", na.rm = TRUE),
    Unsure = sum(Q262 == "Unsure", na.rm = TRUE)
  )

# Print the counts of "Yes", "No", and "Unsure" responses by site for column "Q262"
mpox_uptake_counts_by_site

Q1,Yes,No,Unsure
<chr>,<int>,<int>,<int>
AIDS Service Organization (ASO),40,8,16
Academic hospital/clinic setting,22,10,16
Advocacy organization,0,0,0
Community Based Organization (CBO),27,7,11
Federally Qualified Health Center (FQHC),41,14,22
"Government entity (Local, State, Federal)",31,17,20
Health department (non-clinic),13,8,10
Health department clinic/public health clinic,57,24,29
LBGTQ Health Center/Community Center,17,1,9


## High MPOX vaccine uptake among LGBTQ clients/clients/patients by organization type percentages

In [216]:
# Calculate the percentage of "Yes", "No", and unsure responses for each site 
mpox_uptake_perc_by_site <- final_data %>%
  group_by(Q1) %>%
  summarize(
    Yes_Percentage = sum(Q262 == "Yes", na.rm = TRUE) / n() * 100,
    No_Percentage = sum(Q262 == "No", na.rm = TRUE) / n() * 100,
    Unsure_Percentage = sum(Q262 == "Unsure", na.rm = TRUE) / n() * 100
  )

# Print the percentage of "Yes", "No", and unsure responses for each site
mpox_uptake_perc_by_site

Q1,Yes_Percentage,No_Percentage,Unsure_Percentage
<chr>,<dbl>,<dbl>,<dbl>
AIDS Service Organization (ASO),30.075188,6.015038,12.030075
Academic hospital/clinic setting,27.500000,12.500000,20.000000
Advocacy organization,0.000000,0.000000,0.000000
Community Based Organization (CBO),20.930233,5.426357,8.527132
Federally Qualified Health Center (FQHC),30.597015,10.447761,16.417910
"Government entity (Local, State, Federal)",29.523810,16.190476,19.047619
Health department (non-clinic),31.707317,19.512195,24.390244
Health department clinic/public health clinic,39.041096,16.438356,19.863014
LBGTQ Health Center/Community Center,30.909091,1.818182,16.363636


## Reason for MPOX vaccine hesitancy or refusal counts

In [217]:
mpox_hesitancy_counts <- table(final_data$Q263)
mpox_hesitancy_counts


                    Concerns about vaccine pain/scarring 
                                                      34 
                           Concerns about vaccine safety 
                                                      73 
                              Cultural/religious reasons 
                                                       7 
Lack of knowledge about MPOX transmission and/or vaccine 
                                                     179 
                        Low assumption of infection risk 
                                                     128 
                                        Medical mistrust 
                                                      73 
                                       None of the above 
                                                      40 
                                                   Other 
                                                      31 

## Reason for MPOX vaccine hesitancy or refusal percentages

In [218]:
mpox_hesitancy_perc <- prop.table(mpox_hesitancy_counts)*100
mpox_hesitancy_perc


                    Concerns about vaccine pain/scarring 
                                                6.017699 
                           Concerns about vaccine safety 
                                               12.920354 
                              Cultural/religious reasons 
                                                1.238938 
Lack of knowledge about MPOX transmission and/or vaccine 
                                               31.681416 
                        Low assumption of infection risk 
                                               22.654867 
                                        Medical mistrust 
                                               12.920354 
                                       None of the above 
                                                7.079646 
                                                   Other 
                                                5.486726 

## Contracting MPOX concerns counts

In [219]:
mpox_concern_counts <- table(final_data$Q265)
mpox_concern_counts


    No Unsure    Yes 
   108    118    339 

## Contracting MPOX concerns percentages

In [220]:
mpox_concern_perc <- prop.table(mpox_concern_counts)*100
mpox_concern_perc


      No   Unsure      Yes 
19.11504 20.88496 60.00000 

## Contracting MPOX concerns percentages by prescriber

In [221]:
# Filter data to include only "Yes" responses 
yes_responses_data <- filter(final_data, Q70 == "Yes")

In [222]:
# Calculate the percentage of "Yes," "No," and "Unsure" responses for column "Q265" among prescribers who responded "Yes" in column "Q70"
percentage_concerned_by_prescribers <- yes_responses_data %>%
  summarize(
    Yes_Percentage = sum(Q265 == "Yes", na.rm = TRUE) / n() * 100,
    No_Percentage = sum(Q265 == "No", na.rm = TRUE) / n() * 100,
    Unsure_Percentage = sum(Q265 == "Unsure", na.rm = TRUE) / n() * 100
  )
# Print the percentage of "Yes," "No," and "Unsure" responses for column "Q265" among prescribers who responded "Yes" in column "Q70"
print(percentage_concerned_by_prescribers)

# A tibble: 1 × 3
  Yes_Percentage No_Percentage Unsure_Percentage
           <dbl>         <dbl>             <dbl>
1           43.5          9.09              5.19


## Awareness of HealthHIV MPOX resource center counts

In [223]:
mpox_resource_counts <- table(final_data$Q266)
mpox_resource_counts


 No Yes 
356 209 

## Awareness of HealthHIV MPOX resource center counts

In [224]:
mpox_resource_perc <- prop.table(mpox_resource_counts)*100
mpox_resource_perc


      No      Yes 
63.00885 36.99115 

# Burnout Data

In [225]:
# Considered Quitting
burnout_counts <- table(final_data$Q96)
burnout_counts
burnout_perc <- prop.table(burnout_counts)*100
burnout_perc


 No Yes 
609 495 


      No      Yes 
55.16304 44.83696 

In [226]:
# Emotionally Drained
drain_counts <- table(final_data$Q95)
drain_counts
drain_perc <- prop.table(drain_counts)*100
drain_perc


       A few times a month         A few times a week 
                       258                        208 
A few times a year or less                  Every day 
                       260                         81 
                     Never       Once a month or less 
                        66                        138 
               Once a week 
                        96 


       A few times a month         A few times a week 
                 23.306233                  18.789521 
A few times a year or less                  Every day 
                 23.486902                   7.317073 
                     Never       Once a month or less 
                  5.962060                  12.466125 
               Once a week 
                  8.672087 

In [227]:
#Considered Quitting and Emotionally Drained
burnout_drain <- table(final_data$Q95, final_data$Q96)
burnout_drain
burn_drain_perc <- prop.table(burnout_drain)*100
burn_drain_perc

                            
                              No Yes
  A few times a month        127 130
  A few times a week          57 151
  A few times a year or less 207  52
  Every day                    8  73
  Never                       61   5
  Once a month or less       106  31
  Once a week                 43  53

                            
                                     No        Yes
  A few times a month        11.5036232 11.7753623
  A few times a week          5.1630435 13.6775362
  A few times a year or less 18.7500000  4.7101449
  Every day                   0.7246377  6.6123188
  Never                       5.5253623  0.4528986
  Once a month or less        9.6014493  2.8079710
  Once a week                 3.8949275  4.8007246

## Grouping of Clinical vs. Non-Clinical

In [228]:
# Create a new variable to group clinical roles by provider groups
final_data$prov_grps <- ifelse(final_data$Q66 %in% c('Dentist','Nurse Midwife','Nurse Practicioner','Physician Assistant','Physician'),'Prescribers/Providers',
                               ifelse(final_data$Q66 %in% c('Mental Health Counselor','Mental Health Case Manager','Mental Health Professional','Psychologist','Licensed Clinical Social Worker'),'Mental Health Professional',
                                      ifelse(final_data$Q66 %in% c('Licensed Practical Nurses','Nurse Manager','Registered Nurse','Medical Assistant'),'Nurses/Medical Assistants',
                                             ifelse(final_data$Q66 %in% c('Pharmacist','Phlebotomist','Dietician/Nutrition','Physical Therapist'),'Allied Health Professionals',
                                                    ifelse(final_data$Q66 %in% c('Other Provider/Clinical professional'),'Other',
                                                           'NA')))))

In [229]:
prov_grp_counts <- table(final_data$prov_grps)
prov_grp_counts
prov_grp_perc <- prop.table(prov_grp_counts)*100
prov_grp_perc


Allied Health Professionals  Mental Health Professional 
                         19                          89 
                         NA   Nurses/Medical Assistants 
                        819                         108 
                      Other       Prescribers/Providers 
                          8                          71 


Allied Health Professionals  Mental Health Professional 
                  1.7055655                   7.9892280 
                         NA   Nurses/Medical Assistants 
                 73.5188510                   9.6947935 
                      Other       Prescribers/Providers 
                  0.7181329                   6.3734291 

In [230]:
# Create a new variable to group non-clinical roles by provider groups
final_data$nc_prov_grps <- ifelse(final_data$Q71 %in% c('Administrator','Finance/FiscalManager','Medical Billing'),'Administrators',
                                  ifelse(final_data$Q71 %in% c('Harm Reductionist/Risk Reductionist','Health and Wellness Coach','Health Education Speacialist'),'Health Education and Testing',
                                         ifelse(final_data$Q71 %in% c('Case Manager/Medical Case Manager','Non-clinical social worker','Mental Health Case Manager','Substance Abuse Counselor/Professional','Mental Health Professional'),'Case Managers/Social Workers',
                                                ifelse(final_data$Q71 %in% c('Patient Navigator','Peer Navigator','PrEP Navigator','PrEP Navigator/Coordinator','Health Navigator','Advocate','Caregiver'),'Patient Supporters and Navigators',
                                                       ifelse(final_data$Q71 %in% c('Program Coordinator','HIV Linkage Coordinator','HIV Retention Coordinator','Housing Coordinator','Program Director','Program Manager'),'Program Coordinators/Managers/Directors',
                                                              ifelse(final_data$Q71 %in% c('Outreach Worker','Community Health Worker','HIV Tester','Disease Intervention Specialist'),'Outreach',
                                                                     ifelse(final_data$Q71 %in% c('Clergy/Faith-Based Professional','Researcher','Student'),'Other Non-Clinical Professional',
                                                                            ifelse(final_data$Q71 %in% c('Other Non-clinical Professional'),'Other',
                                                                                   'NA'))))))))

In [231]:
nc_prov_counts <- table(final_data$nc_prov_grps)
nc_prov_counts
nc_prov_perc <- prop.table(nc_prov_counts)*100
nc_prov_perc


                         Administrators            Case Managers/Social Workers 
                                    170                                      15 
           Health Education and Testing                                      NA 
                                     33                                     618 
                                  Other         Other Non-Clinical Professional 
                                     29                                      11 
                               Outreach       Patient Supporters and Navigators 
                                    145                                      67 
Program Coordinators/Managers/Directors 
                                     26 


                         Administrators            Case Managers/Social Workers 
                             15.2603232                               1.3464991 
           Health Education and Testing                                      NA 
                              2.9622980                              55.4757630 
                                  Other         Other Non-Clinical Professional 
                              2.6032316                               0.9874327 
                               Outreach       Patient Supporters and Navigators 
                             13.0161580                               6.0143627 
Program Coordinators/Managers/Directors 
                              2.3339318 

## Cross Tabulations/Chi Square

### Considered Quitting

In [232]:
#Cross tabulation/chi square of Clinical vs Non-Clinical roles x considered quitting
CrossTable(final_data$Q65, final_data$Q96, chisq = TRUE)

#Cross tabulation/chi square of years in role x considered quitting
CrossTable(final_data$Q94, final_data$Q96, chisq = TRUE)

#Cross tabulation/chi square of setting x considered quitting
CrossTable(final_data$Q5, final_data$Q96, chisq = TRUE)

# Create a new subset of urban and rural responses
excluded_set <- c('Suburban','Tribal')
urb_rur_sub <- subset(final_data, !Q5 %in% excluded_set)

#Cross tabulation of urban/rural settings x considered quitting
CrossTable(urb_rur_sub$Q5, urb_rur_sub$Q96, chisq = TRUE)


 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1104 

 
               | final_data$Q96 
final_data$Q65 |        No |       Yes | Row Total | 
---------------|-----------|-----------|-----------|
      Clinical |       196 |       184 |       380 | 
               |     0.885 |     1.089 |           | 
               |     0.516 |     0.484 |     0.344 | 
               |     0.322 |     0.372 |           | 
               |     0.178 |     0.167 |           | 
---------------|-----------|-----------|-----------|
  Non-clinical |       413 |       311 |       724 | 
               |     0.464 |     0.571 |           | 
               |     0.570 |     0.430 |     0.656 | 
               |     0.678 |     0.628 |           | 
               |     0.374 |     0.282 |           | 
---------

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1104 

 
              | final_data$Q96 
final_data$Q5 |        No |       Yes | Row Total | 
--------------|-----------|-----------|-----------|
        Rural |       132 |        94 |       226 | 
              |     0.431 |     0.530 |           | 
              |     0.584 |     0.416 |     0.205 | 
              |     0.217 |     0.190 |           | 
              |     0.120 |     0.085 |           | 
--------------|-----------|-----------|-----------|
     Suburban |       112 |        82 |       194 | 
              |     0.232 |     0.286 |           | 
              |     0.577 |     0.423 |     0.176 | 
              |     0.184 |     0.166 |           | 
              |     0.101 |     0.074 |           | 
--------------|--------

### Emotionally Drained

In [233]:
#Cross tabulation/chi square of Clinical vs Non-Clinical roles x emotionally drained
CrossTable(final_data$Q65, final_data$Q95, chisq = TRUE)

#Cross tabulation/chi square of years in role x emotionally drained
CrossTable(final_data$Q94, final_data$Q95, chisq = TRUE)

#Cross tabulation/chi square of setting x emotionally drained
CrossTable(final_data$Q5, final_data$Q95, chisq = TRUE)

#Cross tabulation of urban/rural settings x emotionally drained
CrossTable(urb_rur_sub$Q5, urb_rur_sub$Q95, chisq = TRUE)


 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1107 

 
               | final_data$Q95 
final_data$Q65 |        A few times a month |         A few times a week | A few times a year or less |                  Every day |                      Never |       Once a month or less |                Once a week |                  Row Total | 
---------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|
      Clinical |                         73 |                         88 |                         93 |                         28 |                         17 |                         51 |                    

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1107 

 
              | final_data$Q95 
final_data$Q5 |        A few times a month |         A few times a week | A few times a year or less |                  Every day |                      Never |       Once a month or less |                Once a week |                  Row Total | 
--------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|
        Rural |                         61 |                         41 |                         55 |                         12 |                         19 |                         27 |                        

### Considered Quitting and Emotionally Drained

In [234]:
chisq.test(burnout_drain)

# Burnout and Drainage within role groups
CrossTable(final_data$prov_grps, final_data$Q95, chisq = TRUE)
CrossTable(final_data$prov_grps, final_data$Q96, chisq = TRUE)
CrossTable(final_data$nc_prov_grps, final_data$Q95, chisq = TRUE)
CrossTable(final_data$nc_prov_grps, final_data$Q96, chisq = TRUE)


	Pearson's Chi-squared test

data:  burnout_drain
X-squared = 268.14, df = 6, p-value < 2.2e-16


Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1107 

 
                            | final_data$Q95 
       final_data$prov_grps |        A few times a month |         A few times a week | A few times a year or less |                  Every day |                      Never |       Once a month or less |                Once a week |                  Row Total | 
----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|
Allied Health Professionals |                          1 |                          2 |                          6 |                          0 |                          

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1104 

 
                            | final_data$Q96 
       final_data$prov_grps |        No |       Yes | Row Total | 
----------------------------|-----------|-----------|-----------|
Allied Health Professionals |        14 |         5 |        19 | 
                            |     1.182 |     1.454 |           | 
                            |     0.737 |     0.263 |     0.017 | 
                            |     0.023 |     0.010 |           | 
                            |     0.013 |     0.005 |           | 
----------------------------|-----------|-----------|-----------|
 Mental Health Professional |        42 |        47 |        89 | 
                            |     1.025 |     1.262 |           | 
                            

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1107 

 
                                        | final_data$Q95 
                final_data$nc_prov_grps |        A few times a month |         A few times a week | A few times a year or less |                  Every day |                      Never |       Once a month or less |                Once a week |                  Row Total | 
----------------------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|----------------------------|
                         Administrators |                         47 |                         27 |                         35 |       

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1104 

 
                                        | final_data$Q96 
                final_data$nc_prov_grps |        No |       Yes | Row Total | 
----------------------------------------|-----------|-----------|-----------|
                         Administrators |        85 |        84 |       169 | 
                                        |     0.726 |     0.893 |           | 
                                        |     0.503 |     0.497 |     0.153 | 
                                        |     0.140 |     0.170 |           | 
                                        |     0.077 |     0.076 |           | 
----------------------------------------|-----------|-----------|-----------|
           Case Managers/Social Workers |        11 | 

# Training topics among clinical providers

In [235]:
# Create a function to get the top 3 topics and their percentages for a given provider group
get_top3_and_percentage <- function(data, topics) {
  # Calculate the counts for each topic within the provider group data
  topic_counts <- sapply(topics, function(topic) sum(data[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

  # Sort the topics based on their counts in descending order
  sorted_topics <- sort(topic_counts, decreasing = TRUE)

  # Get the top 3 topics
  top3_topics <- names(sorted_topics)[1:3]

  # Calculate the total count of providers who provided a ranking (1, 2, or 3) for any of the topics within the provider group data
  total_respondents <- sum(rowSums(!is.na(data[, topics])) > 0)

  # Calculate the percentages for the top 3 topics
  percentages <- (sorted_topics[1:3] / total_respondents) * 100

  # Return the top 3 topics and their percentages
  return(list(top3_topics = top3_topics, percentages = percentages))
}

# Create a list to store results for each provider group
results <- list()

In [236]:
# Create a new variable to group clinical roles by provider groups
final_data$prov_grps <- ifelse(final_data$Q66 %in% c('Dentist', 'Nurse Midwife', 'Nurse Practitioner', 'Physician Assistant', 'Physician'), 'Prescribers/Providers',
                               ifelse(final_data$Q66 %in% c('Mental Health Counselor', 'Mental Health Case Manager', 'Mental Health Professional', 'Psychologist', 'Licensed Clinical Social Worker'), 'Mental Health Professional',
                                      ifelse(final_data$Q66 %in% c('Licensed Practical Nurses', 'Nurse Manager', 'Registered Nurse', 'Medical Assistant'), 'Nurses/Medical Assistants',
                                             ifelse(final_data$Q66 %in% c('Pharmacist', 'Phlebotomist', 'Dietician/Nutrition', 'Physical Therapist'), 'Allied Health Professionals',
                                                    ifelse(final_data$Q66 %in% c('Other Provider/Clinical professional'), 'Other', NA)))))

# Filter data to exclude rows with 'NA' values in the 'prov_grps' column
filtered_data <- final_data[!is.na(final_data$prov_grps), ]

# Filter data to include only rows with provider groups
provider_groups_data <- filtered_data[filtered_data$prov_grps %in% c('Prescribers/Providers', 'Mental Health Professional', 'Nurses/Medical Assistants', 'Allied Health Professionals', 'Other'), ]

## HIV training topics among clinical providers

In [237]:
# Get top 3 topics and percentages for clinical providers
hiv_clinical_results <- get_top3_and_percentage(provider_groups_data, hiv_topics)
hiv_clinical_results

$top3_topics
[1] "Q198" "Q200" "Q199"

$percentages
    Q198     Q200     Q199 
51.62791 44.65116 33.95349

### HIV training topics by clinical provider group

In [238]:
# Get top 3 topics and percentages for each provider group
provider_groups <- c('Prescribers/Providers', 'Mental Health Professional', 'Nurses/Medical Assistants', 'Allied Health Professionals', 'Other')

for (provider_group in provider_groups) {
  group_data <- provider_groups_data[provider_groups_data$prov_grps == provider_group, ]
  result <- get_top3_and_percentage(group_data, hiv_topics)
  results[[provider_group]] <- result
}

# Print the results
for (provider_group in provider_groups) {
  cat("Provider Group:", provider_group, "\n")
  cat("Top 3 Topics:", results[[provider_group]]$top3_topics, "\n")
  cat("Percentages:", results[[provider_group]]$percentages, "\n\n")
}

Provider Group: Prescribers/Providers 
Top 3 Topics: Q198 Q200 Q199 
Percentages: 52.57732 40.20619 35.05155 

Provider Group: Mental Health Professional 
Top 3 Topics: Q204 Q198 Q200 
Percentages: 46.80851 42.55319 42.55319 

Provider Group: Nurses/Medical Assistants 
Top 3 Topics: Q200 Q198 Q199 
Percentages: 54 46 46 

Provider Group: Allied Health Professionals 
Top 3 Topics: Q198 Q200 Q204 
Percentages: 75 56.25 50 

Provider Group: Other 
Top 3 Topics: Q198 Q204 Q203 
Percentages: 100 60 40 



## LGBTQ taining topics among clinical providers

In [239]:
# Get top 3 topics and percentages for clinical providers
lgbtq_clinical_results <- get_top3_and_percentage(provider_groups_data, lgbtq_topics)
lgbtq_clinical_results

$top3_topics
[1] "Q228" "Q230" "Q233"

$percentages
    Q228     Q230     Q233 
44.72843 43.76997 42.17252

### LGBTQ training topics by clinical provider group

In [240]:
# Get top 3 topics and percentages for each provider group
provider_groups <- c('Prescribers/Providers', 'Mental Health Professional', 'Nurses/Medical Assistants', 'Allied Health Professionals', 'Other')

for (provider_group in provider_groups) {
  group_data <- provider_groups_data[provider_groups_data$prov_grps == provider_group, ]
  result <- get_top3_and_percentage(group_data, lgbtq_topics)
  results[[provider_group]] <- result
}

# Print the results
for (provider_group in provider_groups) {
  cat("Provider Group:", provider_group, "\n")
  cat("Top 3 Topics:", results[[provider_group]]$top3_topics, "\n")
  cat("Percentages:", results[[provider_group]]$percentages, "\n\n")
}

Provider Group: Prescribers/Providers 
Top 3 Topics: Q230 Q229 Q228 
Percentages: 45.08197 43.44262 40.98361 

Provider Group: Mental Health Professional 
Top 3 Topics: Q233 Q229 Q230 
Percentages: 50 47.36842 46.05263 

Provider Group: Nurses/Medical Assistants 
Top 3 Topics: Q228 Q234 Q230 
Percentages: 54.34783 48.91304 42.3913 

Provider Group: Allied Health Professionals 
Top 3 Topics: Q229 Q228 Q232 
Percentages: 64.70588 47.05882 41.17647 

Provider Group: Other 
Top 3 Topics: Q230 Q227 Q229 
Percentages: 66.66667 50 50 



## PrEP training topics among clinical providers

In [241]:
# Get top 3 topics and percentages for clinical providers
prep_clinical_results <- get_top3_and_percentage(provider_groups_data, prep_topics)
prep_clinical_results

$top3_topics
[1] "Q214" "Q212" "Q211"

$percentages
    Q214     Q212     Q211 
37.77778 30.22222 29.77778

### PrEP training topics by clinical provider group

In [242]:
# Get top 3 topics and percentages for each provider group
provider_groups <- c('Prescribers/Providers', 'Mental Health Professional', 'Nurses/Medical Assistants', 'Allied Health Professionals', 'Other')

for (provider_group in provider_groups) {
  group_data <- provider_groups_data[provider_groups_data$prov_grps == provider_group, ]
  result <- get_top3_and_percentage(group_data, prep_topics)
  results[[provider_group]] <- result
}

# Print the results
for (provider_group in provider_groups) {
  cat("Provider Group:", provider_group, "\n")
  cat("Top 3 Topics:", results[[provider_group]]$top3_topics, "\n")
  cat("Percentages:", results[[provider_group]]$percentages, "\n\n")
}

Provider Group: Prescribers/Providers 
Top 3 Topics: Q212 Q211 Q214 
Percentages: 38.39286 36.60714 34.82143 

Provider Group: Mental Health Professional 
Top 3 Topics: Q220 Q214 Q219 
Percentages: 43.58974 38.46154 33.33333 

Provider Group: Nurses/Medical Assistants 
Top 3 Topics: Q214 Q218 Q213 
Percentages: 46.42857 33.92857 28.57143 

Provider Group: Allied Health Professionals 
Top 3 Topics: Q212 Q215 Q218 
Percentages: 46.15385 38.46154 30.76923 

Provider Group: Other 
Top 3 Topics: Q224 Q223 Q214 
Percentages: 80 60 40 



# Training topics among non-clinical providers

In [243]:
# Create a new variable to group non-clinical roles by provider groups
nc_prov_grps <- ifelse(final_data$Q71 %in% c('Administrator','Finance/FiscalManager','Medical Billing'),'Administrators',
                                  ifelse(final_data$Q71 %in% c('Harm Reductionist/Risk Reductionist','Health and Wellness Coach','Health Education Speacialist'),'Health Education and Testing',
                                         ifelse(final_data$Q71 %in% c('Case Manager/Medical Case Manager','Non-clinical social worker','Mental Health Case Manager','Substance Abuse Counselor/Professional','Mental Health Professional'),'Case Managers/Social Workers',
                                                ifelse(final_data$Q71 %in% c('Patient Navigator','Peer Navigator','PrEP Navigator','PrEP Navigator/Coordinator','Health Navigator','Advocate','Caregiver'),'Patient Supporters and Navigators',
                                                       ifelse(final_data$Q71 %in% c('Program Coordinator','HIV Linkage Coordinator','HIV Retention Coordinator','Housing Coordinator','Program Director','Program Manager'),'Program Coordinators/Managers/Directors',
                                                              ifelse(final_data$Q71 %in% c('Outreach Worker','Community Health Worker','HIV Tester','Disease Intervention Specialist'),'Outreach',
                                                                     ifelse(final_data$Q71 %in% c('Clergy/Faith-Based Professional','Researcher','Student','Other Non-Clinical Professional'),'Other', NA)))))))
# Filter data to exclude rows with 'NA' values in the 'prov_grps' column
nc_filtered_data <- final_data[!is.na(final_data$nc_prov_grps), ]

# Filter data to include only rows with non-clinical provider groups
nc_provider_groups_data <- final_data[final_data$nc_prov_grps %in% c('Administrators', 'Health Education and Testing', 'Case Managers/Social Workers', 'Patient Supporters and Navigators', 'Program Coordinators/Managers/Directors', 'Outreach', 'Other'), ]

## HIV training topics among non-clinical providers

In [244]:
# Get top 3 topics and percentages for non-clinical providers
hiv_nc_results <- get_top3_and_percentage(nc_provider_groups_data, hiv_topics)
hiv_nc_results

$top3_topics
[1] "Q198" "Q199" "Q205"

$percentages
    Q198     Q199     Q205 
50.82645 43.38843 30.16529

### HIV training topics by non-clinical provider group

In [245]:
# Define the non-clinical provider groups
nc_provider_groups <- c('Administrators', 'Health Education and Testing', 'Case Managers/Social Workers',
                       'Patient Supporters and Navigators', 'Program Coordinators/Managers/Directors',
                       'Outreach', 'Other')

# Get top 3 topics and percentages for each non-clinical provider group
for (nc_provider_group in nc_provider_groups) {
  nc_group_data <- nc_provider_groups_data[nc_provider_groups_data$nc_prov_grps == nc_provider_group, ]
  result <- get_top3_and_percentage(nc_group_data, hiv_topics)
  results[[nc_provider_group]] <- result
}

# Print the results for each non-clinical provider group
for (nc_provider_group in nc_provider_groups) {
  cat("Non-Clinical Provider Group:", nc_provider_group, "\n")
  cat("Top 3 Topics:", results[[nc_provider_group]]$top3_topics, "\n")
  cat("Percentages:", results[[nc_provider_group]]$percentages, "\n\n")
}

Non-Clinical Provider Group: Administrators 
Top 3 Topics: Q198 Q205 Q199 
Percentages: 50.50505 44.44444 41.41414 

Non-Clinical Provider Group: Health Education and Testing 
Top 3 Topics: Q198 Q199 Q200 
Percentages: 66.66667 66.66667 33.33333 

Non-Clinical Provider Group: Case Managers/Social Workers 
Top 3 Topics: Q198 Q199 Q200 
Percentages: 50 50 33.33333 

Non-Clinical Provider Group: Patient Supporters and Navigators 
Top 3 Topics: Q199 Q200 Q198 
Percentages: 46.875 37.5 34.375 

Non-Clinical Provider Group: Program Coordinators/Managers/Directors 
Top 3 Topics: Q205 Q206 Q198 
Percentages: 50 50 42.85714 

Non-Clinical Provider Group: Outreach 
Top 3 Topics: Q198 Q199 Q203 
Percentages: 59.42029 44.92754 30.43478 

Non-Clinical Provider Group: Other 
Top 3 Topics: Q205 Q198 Q206 
Percentages: 53.84615 46.15385 38.46154 



## LGBTQ training topics among non-clinical providers

In [246]:
# Get top 3 topics and percentages for clinical providers
lgbtq_nc_results <- get_top3_and_percentage(nc_provider_groups_data, lgbtq_topics)
lgbtq_nc_results

$top3_topics
[1] "Q234" "Q228" "Q230"

$percentages
    Q234     Q228     Q230 
52.32274 46.94377 45.96577

### LGBTQ training topics by non-clinical provider group

In [247]:
# Get top 3 topics and percentages for each non-clinical provider group
for (nc_provider_group in nc_provider_groups) {
  nc_group_data <- nc_provider_groups_data[nc_provider_groups_data$nc_prov_grps == nc_provider_group, ]
  result <- get_top3_and_percentage(nc_group_data, lgbtq_topics)
  results[[nc_provider_group]] <- result
}

# Print the results for each non-clinical provider group
for (nc_provider_group in nc_provider_groups) {
  cat("Non-Clinical Provider Group:", nc_provider_group, "\n")
  cat("Top 3 Topics:", results[[nc_provider_group]]$top3_topics, "\n")
  cat("Percentages:", results[[nc_provider_group]]$percentages, "\n\n")
}

Non-Clinical Provider Group: Administrators 
Top 3 Topics: Q230 Q234 Q228 
Percentages: 58.9404 49.66887 49.00662 

Non-Clinical Provider Group: Health Education and Testing 
Top 3 Topics: Q230 Q234 Q228 
Percentages: 58.33333 58.33333 45.83333 

Non-Clinical Provider Group: Case Managers/Social Workers 
Top 3 Topics: Q234 Q228 Q230 
Percentages: 66.66667 41.66667 41.66667 

Non-Clinical Provider Group: Patient Supporters and Navigators 
Top 3 Topics: Q234 Q233 Q227 
Percentages: 57.14286 44.89796 38.77551 

Non-Clinical Provider Group: Program Coordinators/Managers/Directors 
Top 3 Topics: Q228 Q234 Q233 
Percentages: 60.86957 52.17391 43.47826 

Non-Clinical Provider Group: Outreach 
Top 3 Topics: Q234 Q228 Q230 
Percentages: 48.0315 44.09449 39.37008 

Non-Clinical Provider Group: Other 
Top 3 Topics: Q234 Q228 Q233 
Percentages: 69.56522 60.86957 39.13043 



## PrEP training topics among non-clinical providers

In [248]:
# Get top 3 topics and percentages for non-clinical providers
prep_nc_results <- get_top3_and_percentage(nc_provider_groups_data, prep_topics)
prep_nc_results

$top3_topics
[1] "Q214" "Q218" "Q211"

$percentages
    Q214     Q218     Q211 
36.26374 34.06593 32.23443

## PrEP training topics by non-clinical provider group

In [249]:
# Get top 3 topics and percentages for each non-clinical provider group
for (nc_provider_group in nc_provider_groups) {
  nc_group_data <- nc_provider_groups_data[nc_provider_groups_data$nc_prov_grps == nc_provider_group, ]
  result <- get_top3_and_percentage(nc_group_data, prep_topics)
  results[[nc_provider_group]] <- result
}

# Print the results for each non-clinical provider group
for (nc_provider_group in nc_provider_groups) {
  cat("Non-Clinical Provider Group:", nc_provider_group, "\n")
  cat("Top 3 Topics:", results[[nc_provider_group]]$top3_topics, "\n")
  cat("Percentages:", results[[nc_provider_group]]$percentages, "\n\n")
}

Non-Clinical Provider Group: Administrators 
Top 3 Topics: Q211 Q218 Q214 
Percentages: 35.39823 30.0885 28.31858 

Non-Clinical Provider Group: Health Education and Testing 
Top 3 Topics: Q214 Q218 Q213 
Percentages: 70 50 40 

Non-Clinical Provider Group: Case Managers/Social Workers 
Top 3 Topics: Q213 Q214 Q212 
Percentages: 42.85714 42.85714 28.57143 

Non-Clinical Provider Group: Patient Supporters and Navigators 
Top 3 Topics: Q214 Q218 Q211 
Percentages: 44.44444 41.66667 25 

Non-Clinical Provider Group: Program Coordinators/Managers/Directors 
Top 3 Topics: Q214 Q218 Q220 
Percentages: 38.88889 33.33333 33.33333 

Non-Clinical Provider Group: Outreach 
Top 3 Topics: Q211 Q214 Q218 
Percentages: 39.47368 38.15789 38.15789 

Non-Clinical Provider Group: Other 
Top 3 Topics: Q220 Q212 Q214 
Percentages: 46.15385 38.46154 38.46154 



# Gilead Needs Assessment Data

## 1. How often do you refer the following complex cases (Q613-Q616)

In [250]:
# Q613 How often do you refer the following complex cases: Choosing an atypical ART regimen in the setting of medication class resistance
atypical_art_counts <- table(final_data$Q613)
atypical_art_counts
atypical_art_perc <- prop.table(atypical_art_counts)*100
atypical_art_perc


All of time time Most of the time None of the time Some of the time 
              41               18              111               41 
          Unsure 
              63 


All of time time Most of the time None of the time Some of the time 
       14.963504         6.569343        40.510949        14.963504 
          Unsure 
       22.992701 

In [251]:
# Q614 How often do you refer the following complex cases: Treating Opportunistic Infections (e.g. toxoplasmosis, tuberculosis, Kaposi's Sarcoma)
opportunistic_infec_counts <- table(final_data$Q614)
opportunistic_infec_counts
opportunistic_infec_perc <- prop.table(opportunistic_infec_counts)*100
opportunistic_infec_perc


All of time time Most of the time None of the time Some of the time 
              48               22               82               67 
          Unsure 
              59 


All of time time Most of the time None of the time Some of the time 
       17.266187         7.913669        29.496403        24.100719 
          Unsure 
       21.223022 

In [252]:
# Q615 How often do you refer the following complex cases: Treating HIV/HCV co-infection
hivhcv_coinfec_counts <- table(final_data$Q615)
hivhcv_coinfec_counts
hivhcv_coinfec_perc <- prop.table(hivhcv_coinfec_counts)*100
hivhcv_coinfec_perc


All of time time Most of the time None of the time Some of the time 
              62               26              102               42 
          Unsure 
              51 


All of time time Most of the time None of the time Some of the time 
       21.908127         9.187279        36.042403        14.840989 
          Unsure 
       18.021201 

In [253]:
# Q616 How often do you refer the following complex cases: HIV treatment in pregnancy
hiv_pregnancy_counts <- table(final_data$Q616)
hiv_pregnancy_counts
hiv_pregnancy_perc <- prop.table(hiv_pregnancy_counts)*100
hiv_pregnancy_perc


All of time time Most of the time None of the time Some of the time 
              86               19               81               33 
          Unsure 
              58 


All of time time Most of the time None of the time Some of the time 
       31.046931         6.859206        29.241877        11.913357 
          Unsure 
       20.938628 

## What do you think are the most timely and important HIV care topics right now (Q618-Q632)

### Total responses

In [254]:
# Create a vector of column names representing the HIV care topic columns (Q617-Q6)
hivcare_topics <- c("Q617", "Q618", "Q619", "Q620", "Q621","Q622","Q623","Q624","Q625","Q626","Q627","Q628","Q629","Q630", "Q630", "Q631")

In [255]:
get_option_counts_and_percentage <- function(data, topics) {
  # Calculate the counts for each option within the data
  option_counts <- sapply(topics, function(topic) sum(data[[topic]] == 1, na.rm = TRUE))
  
  # Sort the topics based on their counts in descending order
  sorted_options <- sort(option_counts, decreasing = TRUE)  

  # Calculate the total count of respondents who provided a response for any of the options within the data
  total_respondents <- sum(rowSums(!is.na(data[, topics])) > 0)

  # Calculate the percentages for each option
  percentages <- (option_counts / total_respondents) * 100

  # Return the counts and percentages as a named list
  return(list(
    option_counts = sorted_options,
    percentages = percentages
  ))
}

In [256]:
# Create a list to store results
hivcare_results <- list()

# Call the 'get_top3_and_percentage' function with the 'final_data' and 'hivcare_topics'
hivcare_results <- get_option_counts_and_percentage(final_data, hivcare_topics)
hivcare_results

$option_counts
Q619 Q617 Q625 Q628 Q618 Q620 Q629 Q626 Q623 Q624 Q621 Q627 Q622 Q630 Q630 Q631 
 403  399  382  331  319  315  299  286  281  250  182  164  143   14   14    3 

$percentages
      Q617       Q618       Q619       Q620       Q621       Q622       Q623 
60.3630862 48.2602118 60.9682300 47.6550681 27.5340393 21.6338880 42.5113464 
      Q624       Q625       Q626       Q627       Q628       Q629       Q630 
37.8214826 57.7912254 43.2677761 24.8108926 50.0756430 45.2344932  2.1180030 
      Q630       Q631 
 2.1180030  0.4538578

### Clinical - overall

In [257]:
# Create a list to store results
hivcare_clinical_results <- list()

# Call the 'get_top3_and_percentage' function with the 'prov_data' and 'hivcare_topics'
hivcare_clinical_results <- get_option_counts_and_percentage(provider_groups_data, hivcare_topics)
hivcare_clinical_results

$option_counts
Q619 Q617 Q625 Q628 Q618 Q629 Q623 Q620 Q624 Q626 Q627 Q621 Q622 Q630 Q630 Q631 
 132  127  118  112  102   92   89   86   84   82   50   45   40    2    2    2 

$percentages
     Q617      Q618      Q619      Q620      Q621      Q622      Q623      Q624 
57.990868 46.575342 60.273973 39.269406 20.547945 18.264840 40.639269 38.356164 
     Q625      Q626      Q627      Q628      Q629      Q630      Q630      Q631 
53.881279 37.442922 22.831050 51.141553 42.009132  0.913242  0.913242  0.913242

### Clincal - by provider type

In [258]:
# Filter data to include only rows with provider groups
provider_groups_data <- final_data[final_data$prov_grps %in% c('Prescribers/Providers', 'Mental Health Professional', 'Nurses/Medical Assistants', 'Allied Health Professionals', 'Other'), ]

# Get top 3 topics and percentages for each clinical provider group
for (provider_group in provider_groups) {
  group_data <- provider_groups_data[provider_groups_data$prov_grps == provider_group, ]
  result <- get_option_counts_and_percentage(group_data, hivcare_topics)

  # Print the results for each clinical provider group
  cat("Provider Group:", provider_group, "\n")
  cat("Option Counts:\n")
  print(result$option_counts)
  cat("\n")
  
  cat("Option Percentages:\n")
  print(result$percentages)
  cat("\n\n")
}


Provider Group: Prescribers/Providers 
Option Counts:
Q617 Q619 Q628 Q618 Q625 Q629 Q623 Q624 Q626 Q620 Q627 Q621 Q622 Q630 Q630 Q631 
  62   54   51   50   46   38   37   37   36   34   27   21   17    1    1    0 

Option Percentages:
     Q617      Q618      Q619      Q620      Q621      Q622      Q623      Q624 
64.583333 52.083333 56.250000 35.416667 21.875000 17.708333 38.541667 38.541667 
     Q625      Q626      Q627      Q628      Q629      Q630      Q630      Q631 
47.916667 37.500000 28.125000 53.125000 39.583333  1.041667  1.041667  0.000000 


Provider Group: Mental Health Professional 
Option Counts:
Q619 Q629 Q620 Q625 Q628 Q626 Q617 Q623 Q618 Q624 Q621 Q622 Q627 Q630 Q630 Q631 
  34   32   29   27   27   24   23   21   16   15    9    6    6    0    0    0 

Option Percentages:
    Q617     Q618     Q619     Q620     Q621     Q622     Q623     Q624 
47.91667 33.33333 70.83333 60.41667 18.75000 12.50000 43.75000 31.25000 
    Q625     Q626     Q627     Q628     Q629     

### Non-clinical - overall

In [259]:
# Create a list to store results
hivcare_results_nc <- list()

# Call the 'get_top3_and_percentage' function with the 'prov_data' and 'hivcare_topics'
hivcare_results_nc <- get_option_counts_and_percentage(nc_provider_groups_data, hivcare_topics)
hivcare_results_nc

$option_counts
Q625 Q617 Q619 Q620 Q628 Q626 Q629 Q618 Q623 Q621 Q624 Q627 Q622 Q630 Q630 Q631 
 170  157  156  138  138  136  122  120  115   93   90   66   58    5    5    1 

$percentages
      Q617       Q618       Q619       Q620       Q621       Q622       Q623 
58.5820896 44.7761194 58.2089552 51.4925373 34.7014925 21.6417910 42.9104478 
      Q624       Q625       Q626       Q627       Q628       Q629       Q630 
33.5820896 63.4328358 50.7462687 24.6268657 51.4925373 45.5223881  1.8656716 
      Q630       Q631 
 1.8656716  0.3731343

### Non-clinical - by provider type

In [260]:
# Filter data to include only rows with non-clinical provider groups
nc_provider_groups_data <- final_data[final_data$nc_prov_grps %in% c('Administrators', 'Health Education and Testing', 'Case Managers/Social Workers', 'Patient Supporters and Navigators', 'Program Coordinators/Managers/Directors', 'Outreach', 'Other'), ]

# Get top 3 topics and percentages for each non-clinical provider group
for (nc_provider_group in nc_provider_groups) {
  nc_group_data <- nc_provider_groups_data[nc_provider_groups_data$nc_prov_grps == nc_provider_group, ]
  result <- get_option_counts_and_percentage(nc_group_data, hivcare_topics)
  
  # Print the results for each clinical provider group
  cat("Provider Group:", nc_provider_group, "\n")
  cat("Option Counts:\n")
  print(result$option_counts)
  cat("\n")
  
  cat("Option Percentages:\n")
  print(result$percentages)
  cat("\n\n")
}

Provider Group: Administrators 
Option Counts:
Q617 Q619 Q625 Q626 Q628 Q629 Q620 Q618 Q623 Q621 Q624 Q627 Q622 Q630 Q630 Q631 
  74   69   64   63   60   56   53   48   43   39   36   24   15    2    2    0 

Option Percentages:
     Q617      Q618      Q619      Q620      Q621      Q622      Q623      Q624 
66.666667 43.243243 62.162162 47.747748 35.135135 13.513514 38.738739 32.432432 
     Q625      Q626      Q627      Q628      Q629      Q630      Q630      Q631 
57.657658 56.756757 21.621622 54.054054 50.450450  1.801802  1.801802  0.000000 


Provider Group: Health Education and Testing 
Option Counts:
Q625 Q620 Q619 Q626 Q621 Q623 Q629 Q618 Q628 Q624 Q617 Q622 Q627 Q630 Q630 Q631 
  11   10    9    8    7    6    6    5    5    4    2    2    2    0    0    0 

Option Percentages:
    Q617     Q618     Q619     Q620     Q621     Q622     Q623     Q624 
15.38462 38.46154 69.23077 76.92308 53.84615 15.38462 46.15385 30.76923 
    Q625     Q626     Q627     Q628     Q629     Q630 

## Where do you currently receive training  (Q163-Q170) check all that apply
### Total responses

In [261]:
# Create a vector of column names representing the current training options 
current_training_locations <- c("Q163", "Q164", "Q165", "Q166", "Q167","Q168","Q169","Q170a","Q170b")
current_training_locations

[1] "Q163"  "Q164"  "Q165"  "Q166"  "Q167"  "Q168"  "Q169"  "Q170a" "Q170b"

In [262]:
# Create a list to store results
current_training_results <- list()

# Call the 'get_option_counts_and_percentage' function with the 'final_data' and 'current_training_locations'
current_training_results <- get_option_counts_and_percentage(final_data, current_training_locations)
current_training_results

$option_counts
 Q167  Q165  Q163  Q166  Q164  Q168 Q170b Q170a  Q169 
  872   811   790   666   642    26     5     3     1 

$percentages
      Q163       Q164       Q165       Q166       Q167       Q168       Q169 
79.7979798 64.8484848 81.9191919 67.2727273 88.0808081  2.6262626  0.1010101 
     Q170a      Q170b 
 0.3030303  0.5050505

## Where do you prefer to receive education/training (Q172-178) check all that apply 
### Total responses

In [263]:
# Create a vector of column names representing the preferred training options 
preferred_training_locations <- c("Q171", "Q172", "Q173", "Q174", "Q175", "Q176", "Q177")

In [264]:
# Create a list to store results
preferred_training_results <- list()

# Call the 'get_option_counts_and_percentage' function with the 'final_data' and 'preferred_training_locations'
preferred_training_results <- get_option_counts_and_percentage(final_data, preferred_training_locations)
preferred_training_results

$option_counts
Q174 Q175 Q171 Q172 Q173 Q176 Q177 
 689  589  585  495  481    9    2 

$percentages
      Q171       Q172       Q173       Q174       Q175       Q176       Q177 
59.0909091 50.0000000 48.5858586 69.5959596 59.4949495  0.9090909  0.2020202

## Which populations do you provide PrEP to (Q646-Q657) check all that apply
### Total responses

In [265]:
# Create a vector of column names representing the population options 
prep_populations <- c("Q646", "Q647", "Q648", "Q649", "Q650","Q651","Q652","Q653","Q654","Q655","Q656","Q657")

In [266]:
# Create a list to store results
prep_populations_results <- list()

# Call the 'get_option_counts_and_percentage' function with the 'final_data' and 'prep_populations'
prep_populations_results <- get_option_counts_and_percentage(final_data, prep_populations)
prep_populations_results

$option_counts
Q654 Q649 Q655 Q648 Q650 Q653 Q652 Q646 Q651 Q647 Q656 Q657 
 258  242  233  223  220  220  200  199  184  161    9    0 

$percentages
     Q646      Q647      Q648      Q649      Q650      Q651      Q652      Q653 
73.703704 59.629630 82.592593 89.629630 81.481481 68.148148 74.074074 81.481481 
     Q654      Q655      Q656      Q657 
95.555556 86.296296  3.333333  0.000000

## Which populations could you be better at reaching 
### Total responses

In [267]:
# Create a vector of column names representing the reaching_populations options 
reaching_populations <- c("Q659", "Q660", "Q661", "Q662", "Q663", "Q664","Q665","Q667","Q668","Q669")

In [268]:
# Create a list to store results
reaching_populations_results <- list()

# Call the 'get_option_counts_and_percentage' function with the 'final_data' and 'reaching_populations'
reaching_populations_results <- get_option_counts_and_percentage(final_data, reaching_populations)
reaching_populations_results

$option_counts
Q659 Q662 Q665 Q668 Q661 Q667 Q664 Q660 Q663 Q669 
  68   28   24   24   23   19   18   17   10    2 

$percentages
      Q659       Q660       Q661       Q662       Q663       Q664       Q665 
26.6666667  6.6666667  9.0196078 10.9803922  3.9215686  7.0588235  9.4117647 
      Q667       Q668       Q669 
 7.4509804  9.4117647  0.7843137

## Top 3 challenges to getting tested for HIV (Q475-Q487)

In [269]:
# Create a vector of column names representing the testing_challenges options
testing_challenges <- c('Q475','Q476','Q477','Q578','Q479','Q480','Q481','Q482','Q483','Q484','Q485','Q486','Q487')

In [270]:
# Get the top 3 challenges
top3_test_chall <- get_top3_and_percentage(final_data, testing_challenges)
top3_test_chall

$top3_topics
[1] "Q475" "Q578" "Q481"

$percentages
    Q475     Q578     Q481 
58.45771 40.54726 39.80100

## Top 3 challenges to timely linking people at risk for HIV to PrEP (Q675-681)

In [271]:
# Create a vector of column names representing the testing_challenges options
linking_challenges <- c('Q675','Q676','Q677','Q678','Q679','Q680','Q681')

In [272]:
# Get the top 3 challenges
top3_link_chall <- get_top3_and_percentage(final_data, linking_challenges)
top3_link_chall

$top3_topics
[1] "Q676" "Q679" "Q675"

$percentages
    Q676     Q679     Q675 
53.70370 52.31481 42.59259

## Top 3 most effective methods for increasing PrEP uptake (Q683-Q692)

In [273]:
# Create a vector of column names representing the method options
prep_methods <- c('Q683','Q684','Q685','Q686','Q687','Q688','Q689','Q690','Q691','Q692','Q693')

In [274]:
# Get the top 3 methods
top3_prep_methods <- get_top3_and_percentage(final_data, prep_methods)
top3_prep_methods

$top3_topics
[1] "Q683" "Q687" "Q684"

$percentages
    Q683     Q687     Q684 
64.06926 43.29004 35.06494

## Top 3 challenges to prescribing PrEP (Q714-Q720)

In [275]:
# Create a vector of column names representing the challenge options
prescribing_challenges <- c('Q714','Q715','Q716','Q717','Q718','Q719','Q720')

In [276]:
# Get the top 3 challenges
top3_prescribing_chall <- get_top3_and_percentage(final_data, prescribing_challenges)
top3_prescribing_chall

$top3_topics
[1] "Q715" "Q714" "Q716"

$percentages
    Q715     Q714     Q716 
60.71429 54.46429 54.46429

## Top 3 challenges to initiating PrEP from the client perspective (Q725-741)

In [277]:
# Create a vector of column names representing the challenge options
initiating_challenges <- c('Q725','Q726','Q727','Q728','Q729','Q730','Q731','Q732','Q733','Q734','Q735','Q736','Q737','Q738','Q739','Q740','Q741')

In [278]:
# Get the top 3 challenges
top3_initiating_chall <- get_top3_and_percentage(final_data, initiating_challenges)
top3_initiating_chall

$top3_topics
[1] "Q726" "Q725" "Q729"

$percentages
    Q726     Q725     Q729 
68.33333 44.16667 37.50000

## Top 3 reasons patients discontinue PrEP (Q749-Q758)

In [279]:
# Create a vector of column names representing the reason options
discontinue_reasons <- c('Q749','Q750','Q751','Q752','Q753','Q754','Q755','Q756','Q757','Q758')

In [280]:
# Get the top 3 reasons
top3_discontinue_reasons <- get_top3_and_percentage(final_data, discontinue_reasons)
top3_discontinue_reasons

$top3_topics
[1] "Q757" "Q750" "Q754"

$percentages
    Q757     Q750     Q754 
58.02469 50.00000 40.12346

## Top 3 ways to improve PrEP services (Q772-Q780) 

In [281]:
# Create a vector of column names representing the ways to improve PrEP services options
improve_services <- c('Q772','Q773','Q774','Q775','Q776','Q777','Q778','Q779','Q780')

In [282]:
# Get the top 3 ways
top3_improve_services <- get_top3_and_percentage(final_data, improve_services)
top3_improve_services

$top3_topics
[1] "Q772" "Q776" "Q774"

$percentages
    Q772     Q776     Q774 
51.24760 42.03455 41.26679

## Top challenges to raising PrEP awareness (Q639-Q644)

In [283]:
# Create a vector of column names representing the challenge options
awareness_challenges <- c('Q639','Q640','Q641','Q642','Q643','Q644')

In [284]:
# Get the top 3 challenges
top3_awareness_chall <- get_top3_and_percentage(final_data, awareness_challenges)
top3_awareness_chall

$top3_topics
[1] "Q640" "Q643" "Q641"

$percentages
    Q640     Q643     Q641 
77.94677 65.77947 61.21673

## HIV testing and treatment challenges

### Top 3 challenges to getting tested (per patient) (Q475-Q487)

In [285]:
# Create a vector of column names representing the testing_challenges options
testing_challenges <- c('Q475','Q476','Q477','Q578','Q479','Q480','Q481','Q482','Q483','Q484','Q485','Q486','Q487')

# Get the top 3 challenges
top3_test_chall <- get_top3_and_percentage(final_data, testing_challenges)
top3_test_chall

$top3_topics
[1] "Q475" "Q578" "Q481"

$percentages
    Q475     Q578     Q481 
58.45771 40.54726 39.80100

### Top 3 challenges to providing testing (per provider) (Q467-Q473)

In [286]:
#Create a vector of column names representing the provider challenge options
prov_test_chall <- c('Q467','Q468','Q469','Q470','Q471','Q472','Q473')

# Get the top 3 challenges
top3_provtest_chall <- get_top3_and_percentage(final_data, prov_test_chall)
top3_provtest_chall

$top3_topics
[1] "Q470" "Q468" "Q467"

$percentages
    Q470     Q468     Q467 
68.01347 57.57576 42.42424

### Top 3 challenges to timely linkage to care (per provider) (Q497-Q505)

In [287]:
# Create a vector of column names representing the provider care linkage challenge options
prov_care_chall <- c('Q497','Q498','Q499','Q500','Q501','Q502','Q503','Q504','Q505','Q505a')

# Get the top 3 challenges
top3_provcare_chall <- get_top3_and_percentage(final_data, prov_care_chall)
top3_provcare_chall

$top3_topics
[1] "Q503" "Q500" "Q502"

$percentages
    Q503     Q500     Q502 
51.80723 48.49398 44.57831

### Top 3 challenges to timely linking to care (per patient) (Q506-522)

In [288]:
# Create a vector of column names representing the patient care linkage challenge options
pat_care_chall <- c('Q506','Q507','Q508','Q509','Q510','Q511','Q512','Q513','Q514','Q515','Q516','Q517','Q518','Q519','Q520','Q521','Q522')

# Get the top 3 challenges
top3_patcare_chall <- get_top3_and_percentage(final_data, pat_care_chall)
top3_patcare_chall

$top3_topics
[1] "Q506" "Q508" "Q515"

$percentages
    Q506     Q508     Q515 
61.40351 40.64327 26.60819

### Top 3 challenges to starting people on ART (provider) (Q529-Q535)

In [289]:
# Create a vector of column names representing the patient care linkage challenge options
art_chall <- c('Q529','Q530','Q531','Q532','Q533','Q534','Q535')

# Get the top 3 challenges
top3_art_chall <- get_top3_and_percentage(final_data, art_chall)
top3_art_chall

$top3_topics
[1] "Q531" "Q533" "Q529"

$percentages
    Q531     Q533     Q529 
64.28571 62.75510 55.61224

### Top 3 challenges to initiating ART (patient) (Q537-Q546)

In [290]:
# Create a vector of column names representing the patient ART linkage challenge challenge options
pat_art_chall <- c('Q537','Q538','Q539','Q540','Q541','Q542','Q543','Q544','Q545','Q546')

# Get the top 3 challenges
top3_patart_chall <- get_top3_and_percentage(final_data, pat_art_chall)
top3_patart_chall

$top3_topics
[1] "Q540" "Q537" "Q539"

$percentages
    Q540     Q537     Q539 
57.21393 44.77612 42.78607

### Top 3 challenges to keeping appts (Q557-Q572)

In [291]:
# Create a vector of column names representing the appointment adherence challenge options
apts_adh_chall <- c('Q557','Q558','Q559','Q560','Q561','Q562','Q563','Q564','Q565','Q566','Q567','Q568','Q569','Q570','Q571','Q572')

# Get the top 3 challenges
top3_aptsadh_chall <- get_top3_and_percentage(final_data, apts_adh_chall)
top3_aptsadh_chall

$top3_topics
[1] "Q560" "Q562" "Q561"

$percentages
    Q560     Q562     Q561 
49.77169 45.20548 37.44292

### Top 3 challenges to adhere to ART (Q591-Q602)

In [292]:
# Create a vector of column names representing the appointment adherence challenge options
art_adh_chall <- c('Q591','Q592','Q593','Q594','Q595','Q596','Q597','Q598','Q599','Q600','Q601','Q602')

# Get the top 3 challenges
top3_artadh_chall <- get_top3_and_percentage(final_data, art_adh_chall)
top3_artadh_chall

$top3_topics
[1] "Q594" "Q596" "Q593"

$percentages
    Q594     Q596     Q593 
64.25339 51.13122 46.15385

## Workforce Challenges 

### Which of the following workforce issues are you currently facing? (Q98-Q106 & 107a-107d) rank order

In [293]:
# Create a vector of column names representing the workforce issue options
workforce_issues <- c('Q98','Q99','Q100','Q101','Q102','Q103','Q104','Q105','Q106','Q107a','Q107b','Q107c','Q107d')

# Get the top 3 issues
workforce_issues_results <- get_top3_and_percentage(final_data, workforce_issues)
workforce_issues_results

$top3_topics
[1] "Q98"  "Q100" "Q105"

$percentages
     Q98     Q100     Q105 
53.41797 52.05078 51.07422

### What is your organization doing to address workforce challenges (Q108-Q116 & 117a-117b)

In [294]:
# Create a vector of column names representing the options
address_workforce_issues <- c('Q108','Q109','Q110','Q111','Q112','Q113','Q114','Q115','Q116','Q117a','Q117b')

# Get the top 3 issues
address_workforce_issues_results <- get_top3_and_percentage(final_data, address_workforce_issues)
address_workforce_issues_results

$top3_topics
[1] "Q109" "Q112" "Q111"

$percentages
    Q109     Q112     Q111 
37.32970 31.60763 31.24432

## Policy Questions frequencies/percentages

### Are you aware of any HIV criminalization laws in your state or region? (Q784)

In [295]:
law_aware_count <- table(final_data$Q784)
law_aware_count

law_aware_perc <- prop.table(law_aware_count)*100
law_aware_perc


 No Yes 
465 311 


      No      Yes 
59.92268 40.07732 

### Have you observed any differences in how HIV criminalization affects your patients who belong to marginalized communities, such as people of color, LGBTQ+ individuals, or people who use drugs? (Q787)

In [296]:
crim_diff_count <- table(final_data$Q787)
crim_diff_count

crim_diff_perc <- prop.table(crim_diff_count)*100
crim_diff_perc


 No Yes 
150 162 


      No      Yes 
48.07692 51.92308 

### Do you think that HIV criminalization policies have a negative impact on public health efforts to prevent HIV transmission? (Q791)

In [297]:
crim_impact_count <- table(final_data$Q791)
crim_impact_count

crim_impact_perc <- prop.table(crim_impact_count)*100
crim_impact_perc


 No Yes 
203 573 


      No      Yes 
26.15979 73.84021 

### Are you familiar with HHS guidelines on zero-cost sharing and co-pays for PrEP? (Q807)

In [298]:
hhs_guide_count <- table(final_data$Q807)
hhs_guide_count

hhs_guide_perc <- prop.table(hhs_guide_count)*100
hhs_guide_perc


 No Yes 
313 253 


      No      Yes 
55.30035 44.69965 

### Are you aware of any anti-LGBTQ healthcare bills in your state or region? (Q808)

In [299]:
lgbtq_bills_count <- table(final_data$Q808)
lgbtq_bills_count

lgbtq_bills_perc <- prop.table(lgbtq_bills_count)*100
lgbtq_bills_perc


 No Yes 
487 347 


      No      Yes 
58.39329 41.60671 

### Do you think these policies impact the health of your clients/clients/patients? (Q809)

In [300]:
bills_impact_count <- table(final_data$Q809)
bills_impact_count

bills_impact_perc <- prop.table(bills_impact_count)*100
bills_impact_perc


 No Yes 
 20 329 


       No       Yes 
 5.730659 94.269341 

### In your experience, has the recent escalation of anti-trans healthcare legislation across the county (e.g. banning gender-affirming medical care) negatively impacted the mental health of your transgender and/or gender non-conforming clients/patients? (Q811)

In [301]:
transleg_impact_count <- table(final_data$Q811)
transleg_impact_count

transleg_impact_perc <- prop.table(transleg_impact_count)*100
transleg_impact_perc


    No Unsure    Yes 
    73    276    486 


       No    Unsure       Yes 
 8.742515 33.053892 58.203593 

### Have you ever been prevented from providing healthcare to LGBTQ+ clients/patients due to local legislation? (Q813)

In [302]:
lgbt_prev_count <- table(final_data$Q813)
lgbt_prev_count

lgbt_prev_perc <- prop.table(lgbt_prev_count)*100
lgbt_prev_perc


 No Yes 
800  35 


       No       Yes 
95.808383  4.191617 

### Are you aware of any harm reduction criminalization policies or laws in your region or state? (Q815)

In [303]:
hrpol_count <- table(final_data$Q815)
hrpol_count

hrpol_perc <- prop.table(hrpol_count)*100
hrpol_perc


 No Yes 
505 187 


      No      Yes 
72.97688 27.02312 

### Do you think these policies impact the health of your clients/patients? (Q816)

In [304]:
hrpol_impact_count <- table(final_data$Q816)
hrpol_impact_count

hrpol_impact_perc <- prop.table(hrpol_impact_count)*100
hrpol_impact_perc


 No Yes 
  9 178 


       No       Yes 
 4.812834 95.187166 

# Gender Affirming Care

## Do you personally provide gender services? Q326 yes/no

In [305]:
gen_care_count <- table(final_data$Q326)
gen_care_count

gen_care_perc <- prop.table(gen_care_count)*100
gen_care_perc


 No Yes 
167 128 


      No      Yes 
56.61017 43.38983 

## Gender affirming environments

### Forms that include gender identity and sex at birth

In [306]:
forms_genid_count <- table(final_data$Q144)
forms_genid_count

forms_genid_perc <- prop.table(forms_genid_count)*100
forms_genid_perc


    No Unsure    Yes 
    75    105    919 


       No    Unsure       Yes 
 6.824386  9.554140 83.621474 

### Forms that include pronoun field

In [307]:
forms_pronoun_count <- table(final_data$Q145)
forms_pronoun_count

forms_pronoun_perc <- prop.table(forms_pronoun_count)*100
forms_pronoun_perc


    No Unsure    Yes 
   198    148    755 


      No   Unsure      Yes 
17.98365 13.44233 68.57402 

### Forms that include preferred or chosen name field

In [308]:
forms_name_count <- table(final_data$Q146)
forms_name_count

forms_name_perc <- prop.table(forms_pronoun_count)*100
forms_name_perc


    No Unsure    Yes 
   131    106    860 


      No   Unsure      Yes 
17.98365 13.44233 68.57402 

### Forms that include sexual orientation fields

In [309]:
forms_orient_count <- table(final_data$Q147)
forms_orient_count

forms_orient_perc <- prop.table(forms_orient_count)*100
forms_orient_perc


    No Unsure    Yes 
    97    103    900 


       No    Unsure       Yes 
 8.818182  9.363636 81.818182 

### Gender neutral bathrooms

In [310]:
bathroom_count <- table(final_data$Q148)
bathroom_count

bathroom_perc <- prop.table(bathroom_count)*100
bathroom_perc


    No Unsure    Yes 
   340     66    695 


      No   Unsure      Yes 
30.88102  5.99455 63.12443 

### Educational and promotional materials that are inclusive of all genders and gender expression (e.g. posters, brochures, palm cards, etc.)

In [311]:
material_gen_count <- table(final_data$Q149)
material_gen_count

material_gen_perc <- prop.table(material_gen_count)*100
material_gen_perc


    No Unsure    Yes 
   151    132    814 


      No   Unsure      Yes 
13.76481 12.03282 74.20237 

### Educational materials that are inclusive of all races and ethnicities (e.g. posters, brochures, palm cards, etc.)

In [312]:
material_race_count <- table(final_data$Q150)
material_race_count

material_race_perc <- prop.table(material_race_count)*100
material_race_perc


    No Unsure    Yes 
    88     73    937 


       No    Unsure       Yes 
 8.014572  6.648452 85.336976 

## Gender affirming environments x type of organization

In [313]:
CrossTable(final_data$Q144, final_data$Q1, chisq = TRUE)

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1099 

 
                | final_data$Q1 
final_data$Q144 |              Academic hospital/clinic setting |                         Advocacy organization |               AIDS Service Organization (ASO) |            Community Based Organization (CBO) |      Federally Qualified Health Center (FQHC) |     Government entity (Local, State, Federal) |                Health department (non-clinic) | Health department clinic/public health clinic |          LBGTQ Health Center/Community Center |                       Non-profit organization |                                         Other |                                      Pharmacy |               Private hospital/clinic setting |                              Private practice |              Substa

In [314]:
CrossTable(final_data$Q145, final_data$Q1, chisq = TRUE)

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1101 

 
                | final_data$Q1 
final_data$Q145 |              Academic hospital/clinic setting |                         Advocacy organization |               AIDS Service Organization (ASO) |            Community Based Organization (CBO) |      Federally Qualified Health Center (FQHC) |     Government entity (Local, State, Federal) |                Health department (non-clinic) | Health department clinic/public health clinic |          LBGTQ Health Center/Community Center |                       Non-profit organization |                                         Other |                                      Pharmacy |               Private hospital/clinic setting |                              Private practice |              Substa

In [315]:
CrossTable(final_data$Q146, final_data$Q1, chisq = TRUE)

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1097 

 
                | final_data$Q1 
final_data$Q146 |              Academic hospital/clinic setting |                         Advocacy organization |               AIDS Service Organization (ASO) |            Community Based Organization (CBO) |      Federally Qualified Health Center (FQHC) |     Government entity (Local, State, Federal) |                Health department (non-clinic) | Health department clinic/public health clinic |          LBGTQ Health Center/Community Center |                       Non-profit organization |                                         Other |                                      Pharmacy |               Private hospital/clinic setting |                              Private practice |              Substa

In [316]:
CrossTable(final_data$Q147, final_data$Q1, chisq = TRUE)

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1100 

 
                | final_data$Q1 
final_data$Q147 |              Academic hospital/clinic setting |                         Advocacy organization |               AIDS Service Organization (ASO) |            Community Based Organization (CBO) |      Federally Qualified Health Center (FQHC) |     Government entity (Local, State, Federal) |                Health department (non-clinic) | Health department clinic/public health clinic |          LBGTQ Health Center/Community Center |                       Non-profit organization |                                         Other |                                      Pharmacy |               Private hospital/clinic setting |                              Private practice |              Substa

In [317]:
CrossTable(final_data$Q148, final_data$Q1, chisq = TRUE)

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1101 

 
                | final_data$Q1 
final_data$Q148 |              Academic hospital/clinic setting |                         Advocacy organization |               AIDS Service Organization (ASO) |            Community Based Organization (CBO) |      Federally Qualified Health Center (FQHC) |     Government entity (Local, State, Federal) |                Health department (non-clinic) | Health department clinic/public health clinic |          LBGTQ Health Center/Community Center |                       Non-profit organization |                                         Other |                                      Pharmacy |               Private hospital/clinic setting |                              Private practice |              Substa

In [318]:
CrossTable(final_data$Q149, final_data$Q1, chisq = TRUE)

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1097 

 
                | final_data$Q1 
final_data$Q149 |              Academic hospital/clinic setting |                         Advocacy organization |               AIDS Service Organization (ASO) |            Community Based Organization (CBO) |      Federally Qualified Health Center (FQHC) |     Government entity (Local, State, Federal) |                Health department (non-clinic) | Health department clinic/public health clinic |          LBGTQ Health Center/Community Center |                       Non-profit organization |                                         Other |                                      Pharmacy |               Private hospital/clinic setting |                              Private practice |              Substa

In [319]:
CrossTable(final_data$Q150, final_data$Q1, chisq = TRUE)

Warning message in chisq.test(t, correct = FALSE, ...):
“Chi-squared approximation may be incorrect”



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  1098 

 
                | final_data$Q1 
final_data$Q150 |              Academic hospital/clinic setting |                         Advocacy organization |               AIDS Service Organization (ASO) |            Community Based Organization (CBO) |      Federally Qualified Health Center (FQHC) |     Government entity (Local, State, Federal) |                Health department (non-clinic) | Health department clinic/public health clinic |          LBGTQ Health Center/Community Center |                       Non-profit organization |                                         Other |                                      Pharmacy |               Private hospital/clinic setting |                              Private practice |              Substa

## Gender affirming care services offered to what age ranges (Q336-Q340)

In [320]:
# Create a vector of column names representing the ages 
gen_affirm_ages <- c('Q336','Q337','Q338','Q339','Q340')

In [321]:
# Create a list to store results
gen_affirm_ages_results <- list()

# Call the 'get_option_counts_and_percentage' function with the 'final_data' and 'gen_affirm_services'
gen_affirm_ages_results <- get_option_counts_and_percentage(final_data, gen_affirm_ages)
gen_affirm_ages_results

$option_counts
Q338 Q337 Q339 Q336 Q340 
 118  112  101   29    2 

$percentages
    Q336     Q337     Q338     Q339     Q340 
22.65625 87.50000 92.18750 78.90625  1.56250

## Gender affirming facilitators Q342-Q348 rank order

In [322]:
# Create a vector of column names representing the facilitator options
gen_affirm_facilitators <- c('Q342','Q343','Q344','Q345','Q346','Q347','Q348')

In [323]:
# Create a list to store results
gen_affirm_facilitators_results <- list()

# Call the 'get_top3_and_percentage' function with the 'final_data' and 'gen_affirm_facilitators'
gen_affirm_facilitators_results <- get_top3_and_percentage(final_data, gen_affirm_facilitators)
gen_affirm_facilitators_results

$top3_topics
[1] "Q342" "Q347" "Q343"

$percentages
    Q342     Q347     Q343 
85.82677 62.99213 42.51969

## Gender affirming barriers Q350-Q358

In [324]:
# Create a vector of column names representing the barrier options
gen_provider_barriers <- c('Q350','Q351','Q352','Q353','Q354','Q355','Q356','Q357','Q358')

In [325]:
# Create a list to store results
gen_provider_barriers_results <- list()

# Call the 'get_top3_and_percentage' function with the 'final_data' and 'gen_provider_barriers'
gen_provider_barriers_results <- get_top3_and_percentage(final_data, gen_provider_barriers)
gen_provider_barriers_results

$top3_topics
[1] "Q350" "Q351" "Q353"

$percentages
Q350 Q351 Q353 
82.4 41.6 29.6

## Gender affirming patient barriers Q360-Q373

In [326]:
# Create a vector of column names representing the barrier options
gen_patient_barriers <- c('Q360','Q361','Q362','Q363','Q364','Q365','Q366','Q367','Q368','Q369','Q370','Q371','Q372','Q373')

In [327]:
# Create a list to store results
gen_patient_barriers_results <- list()

# Call the 'get_top3_and_percentage' function with the 'final_data' and 'gen_patient_barriers'
gen_patient_barriers_results <- get_top3_and_percentage(final_data, gen_patient_barriers)
gen_patient_barriers_results


$top3_topics
[1] "Q360" "Q364" "Q363"

$percentages
    Q360     Q364     Q363 
64.28571 46.03175 41.26984

## Facilitators to LGBTQ services Q279-286 rank order

In [328]:
# Create a vector of column names representing the facilitator options
lgbtq_facilitators <- c('Q279','Q280','Q281','Q282','Q283','Q284','Q285','Q286')

In [329]:
# Create a list to store results
lgbtq_facilitators_results <- list()

# Call the 'get_top3_and_percentage' function with the 'final_data' and 'lgbtq_facilitators'
lgbtq_facilitators_results <- get_top3_and_percentage(final_data, lgbtq_facilitators)
lgbtq_facilitators_results

$top3_topics
[1] "Q282" "Q279" "Q285"

$percentages
    Q282     Q279     Q285 
64.38356 54.24658 47.12329

## Barriers to providing LGBTQ services Q288-Q296 & Q297a

In [330]:
# Create a vector of column names representing the barrier options
lgbtq_provider_barriers <- c('Q288','Q289','Q290','Q291','Q292','Q293','Q294','Q295','Q296','Q297a')

In [331]:
# Create a list to store results
lgbtq_provider_barriers_results <- list()

# Call the 'get_top3_and_percentage' function with the 'final_data' and 'lgbtq_provider_barriers'
lgbtq_provider_barriers_results <- get_top3_and_percentage(final_data, lgbtq_provider_barriers)
lgbtq_provider_barriers_results

$top3_topics
[1] "Q288" "Q289" "Q291"

$percentages
    Q288     Q289     Q291 
67.03911 44.97207 34.63687

## Barriers to accessing LGBTQ services Q298-Q312

In [332]:
# Create a vector of column names representing the barrier options
lgbtq_patient_barrier <- c('Q298','Q299','Q300','Q301','Q302','Q303','Q304','Q305','Q306','Q307','Q308','Q309','Q310','Q311','Q312')

In [333]:
# Create a list to store results
lgbtq_patient_barrier_results <- list()

# Call the 'get_top3_and_percentage' function with the 'final_data' and 'lgbtq_patient_barrier'
lgbtq_patient_barrier_results <- get_top3_and_percentage(final_data, lgbtq_patient_barrier)
lgbtq_patient_barrier_results

$top3_topics
[1] "Q298" "Q302" "Q306"

$percentages
    Q298     Q302     Q306 
66.02210 46.40884 31.49171

# Counts/Percentages of all

## Which of the following groups LGBTQ serve Q127-134

In [334]:
lgbtq_serve_groups <- c('Q127','Q128','Q129','Q130','Q131','Q132','Q133','Q134')

In [335]:
# Create a list to store results
lgbtq_serve_results <- list()

# Call the 'get_option_counts_and_percentage' function with the 'final_data' and 'lgbtq_serve_groups'
lgbtq_serve_results <- get_option_counts_and_percentage(final_data, lgbtq_serve_groups)
lgbtq_serve_results

$option_counts
Q133 Q127 Q129 Q130 Q132 Q131 Q128 Q134 
1003   83   71   70   57   56   53   12 

$percentages
     Q127      Q128      Q129      Q130      Q131      Q132      Q133      Q134 
 7.511312  4.796380  6.425339  6.334842  5.067873  5.158371 90.769231  1.085973

## Does your organization provide services to any of the following populations? Select all Q135-143

In [336]:
provide_populations <- c('Q135','Q136','Q137','Q138','Q139','Q140','Q141','Q142','Q143')

In [337]:
# Create a list to store results
provide_populations_results <- list()
# Call the 'get_option_counts_and_percentage' function with the 'final_data' and 'provide_populations'
provide_populations_results <- get_option_counts_and_percentage(final_data, provide_populations)
provide_populations_results

$option_counts
Q137 Q135 Q139 Q142 Q136 Q140 Q141 Q138 Q143 
1013 1002  939  931  927  877  860  836   15 

$percentages
     Q135      Q136      Q137      Q138      Q139      Q140      Q141      Q142 
90.843155 84.043518 91.840435 75.793291 85.131460 79.510426 77.969175 84.406165 
     Q143 
 1.359927

## What do you think are the most significant facilitators to providing services to LGBTQ+ clients/patients? Q279-286 rank top 3 

In [338]:
# already calculated
lgbtq_facilitators_results

$top3_topics
[1] "Q282" "Q279" "Q285"

$percentages
    Q282     Q279     Q285 
64.38356 54.24658 47.12329

## Health care topics rank order (Q179-Q196 & Q197a-Q197b)

In [339]:
health_care_topics <- c('Q179','Q180','Q181','Q182','Q183','Q184','Q185','Q186','Q187','Q188','Q189','Q190','Q191','Q192','Q193','Q194','Q195','Q196','Q197a','Q197b')

In [340]:
# Create a list to store results
health_care_topics_results <- list()

# Call the 'get_top3_and_percentage' function with the 'final_data' and 'health_care_topics'
health_care_topics_results <- get_top3_and_percentage(final_data, health_care_topics)
health_care_topics_results

$top3_topics
[1] "Q179" "Q183" "Q186"

$percentages
    Q179     Q183     Q186 
28.19979 24.76587 23.62123

# HIV Care Questions 

## Do you know how many tested for HIV last year Q489

In [341]:
know_hiv_testing <- table(final_data$Q489)
know_hiv_testing
kow_hiv_testing_perc <- prop.table(know_hiv_testing)*100
kow_hiv_testing_perc


 No Yes 
215 114 


      No      Yes 
65.34954 34.65046 

### Provide number of HIV tested last year Q490

In [342]:
hiv_testing_number <- table(final_data$Q490)
hiv_testing_number
hiv_testing_perc <- prop.table(hiv_testing_number)*100
hiv_testing_perc


      <50 1000-1999      2000   501-999    51-500 
        3        22        21        24        44 


      <50 1000-1999      2000   501-999    51-500 
 2.631579 19.298246 18.421053 21.052632 38.596491 

### How has number changed from last year Q491

In [343]:
hiv_testing_change <- table(final_data$Q491)
hiv_testing_change
hiv_testing_change_perc <- prop.table(hiv_testing_change)*100
hiv_testing_change_perc


Decreased Increased No change    Unsure 
       18        60        18        18 


Decreased Increased No change    Unsure 
 15.78947  52.63158  15.78947  15.78947 

## Do you know how many diagnosed with HIV last year Q492

In [344]:
know_hiv_dignosis<- table(final_data$Q492)
know_hiv_dignosis
know_hiv_dignosis_perc <- prop.table(know_hiv_dignosis)*100
know_hiv_dignosis_perc


 No Yes 
212 117 


      No      Yes 
64.43769 35.56231 

### Provide number of patients diagnosed with HIV last year Q493

In [345]:
hiv_diagnosis_number <- table(final_data$Q493)
hiv_diagnosis_number
hiv_diagnosis_perc <- prop.table(hiv_diagnosis_number)*100
hiv_diagnosis_perc


      <50 1000-1999      2000   501-999    51-500 
       94         1         2         1        19 


       <50  1000-1999       2000    501-999     51-500 
80.3418803  0.8547009  1.7094017  0.8547009 16.2393162 

### How has number changed from last year Q494

In [346]:
hiv_diagnosis_change <- table(final_data$Q494)
hiv_diagnosis_change
hiv_diagnosis_change_perc <- prop.table(hiv_diagnosis_change)*100
hiv_diagnosis_change_perc


Decreased Increased No change    Unsure 
       13        42        35        27 


Decreased Increased No change    Unsure 
 11.11111  35.89744  29.91453  23.07692 

### Why number changed Q495

In [347]:
### open response???

## Do you know how many referrals for HIV past year?  Q524

In [348]:
know_hiv_referrals <- table(final_data$Q524)
know_hiv_referrals
know_hiv_referrals_perc <- prop.table(know_hiv_referrals)*100
know_hiv_referrals_perc


 No Yes 
264  92 


     No     Yes 
74.1573 25.8427 

## Please indicate how many people your organization referred to HIV care last year Q525

In [349]:
hiv_referrals_number <- table(final_data$Q525)
hiv_referrals_number
hiv_referrals_perc <- prop.table(hiv_referrals_number)*100
hiv_referrals_perc


      <50 1000-1999      2000   501-999    51-500 
       52         2         1         2        36 


      <50 1000-1999      2000   501-999    51-500 
55.913978  2.150538  1.075269  2.150538 38.709677 

## How has the number of people your organization referred to HIV care last year changed? Q526

In [350]:
hiv_referral_change <- table(final_data$Q526)
hiv_referral_change
hiv_referral_change_perc <- prop.table(hiv_referral_change)*100
hiv_referral_change_perc


Decreased Increased No change    Unsure 
        6        41        28        18 


Decreased Increased No change    Unsure 
 6.451613 44.086022 30.107527 19.354839 

### Why do you think the number of people referred to HIV care changed? Q527

In [351]:
## open response??

## Do you know how many people your organization provided HIV services to last year? Q549

In [352]:
know_hiv_services <- table(final_data$Q549)
know_hiv_services
know_hiv_services_perc <- prop.table(know_hiv_services)*100
know_hiv_services_perc


 No Yes 
110 100 


      No      Yes 
52.38095 47.61905 

### Please indicate how many people your organization provided HIV services to last year Q550

In [353]:
hiv_services_number <- table(final_data$Q550)
hiv_services_number
hiv_services_perc <- prop.table(hiv_services_number)*100
hiv_services_perc 


      <50 1000-1999      2000   501-999    51-500 
       13        21         3        19        44 


      <50 1000-1999      2000   501-999    51-500 
       13        21         3        19        44 

### How has the number of people your organization provided HIV services to changed from last year? Q551

In [354]:
hiv_services_change <- table(final_data$Q551)
hiv_services_change
hiv_services_change_perc <- prop.table(hiv_services_change)*100
hiv_services_change_perc


Decreased Increased No change    Unsure 
        5        53        27        15 


Decreased Increased No change    Unsure 
        5        53        27        15 

### Why do you think the number of people receiving HIV services from your organization changed? Q552

In [355]:
### open response???

## Approximately what percentage of your patients rely on Ryan White/ADAP for ARTs? Q553

In [356]:
rw_adap_counts <- table(final_data$Q553)
rw_adap_counts
rw_adap_perc <- prop.table(rw_adap_counts)*100
rw_adap_perc


               <49%              50-59%              60-69%              70-79% 
                 40                  21                  11                  14 
             80-89%              90-99% None/not applicable              Unsure 
                 33                  39                   8                  44 


               <49%              50-59%              60-69%              70-79% 
          19.047619           10.000000            5.238095            6.666667 
             80-89%              90-99% None/not applicable              Unsure 
          15.714286           18.571429            3.809524           20.952381 

## Which health insurance type generally requires the most time to complete and receive a determination for ART? Q554

In [357]:
longest_ins <- table(final_data$Q554)
longest_ins
longest_ins_perc <- prop.table(longest_ins)*100
longest_ins_perc


                    Medicaid                     Medicare 
                          38                            5 
      Other public insurance Private insurance (ACA plan) 
                           4                           16 
Private insurance (employer)              Ryan White/ADAP 
                          47                           19 
      There is no difference                       Unsure 
                          10                           71 


                    Medicaid                     Medicare 
                   18.095238                     2.380952 
      Other public insurance Private insurance (ACA plan) 
                    1.904762                     7.619048 
Private insurance (employer)              Ryan White/ADAP 
                   22.380952                     9.047619 
      There is no difference                       Unsure 
                    4.761905                    33.809524 

## What other health insurance plan requires the most time to complete and receive a determination for ART? Q555

In [358]:
other_longest_ins <- table(final_data$Q555)
other_longest_ins

< table of extent 0 >

## Does your organization provide any of the following retention in care strategies? Q574-Q583

In [359]:
ret_strategies <- c('Q574','Q575','Q576','Q577','Q578','Q579','Q580','Q581','Q582','Q583')

In [360]:
# Create a list to store results
ret_strategies_results <- list()
# Call the 'get_option_counts_and_percentage' function with the 'final_data' and 'ret_strategies'
ret_strategies_results <- get_option_counts_and_percentage(final_data, ret_strategies)
ret_strategies_results

$option_counts
Q575 Q574 Q577 Q576 Q578 Q580 Q581 Q579 Q582 Q583 
 204  198  191  189  163  149  119  108    6    2 

$percentages
     Q574      Q575      Q576      Q577      Q578      Q579      Q580      Q581 
86.842105 89.473684 82.894737 83.771930 71.491228 47.368421 65.350877 52.192982 
     Q582      Q583 
 2.631579  0.877193

## Are you aware of your organization's lost to care rate? Q586

In [361]:
know_ltcr <- table(final_data$Q586)
know_ltcr
know_ltcr_perc <- prop.table(know_ltcr)*100
know_ltcr_perc


 No Yes 
199  30 


      No      Yes 
86.89956 13.10044 

### What is your organization's lost to care rate? Q587

In [362]:
ltcr_numbers <- table(final_data$Q587)
ltcr_numbers
ltcr_perc <- prop.table(ltcr_numbers)*100
ltcr_perc


0.50%    0%    1%   10%   119   15%   23%   25%   30%    5%    7%    8% 
    1     1     1     3     1     7     1     1     3     8     1     2 


    0.50%        0%        1%       10%       119       15%       23%       25% 
 3.333333  3.333333  3.333333 10.000000  3.333333 23.333333  3.333333  3.333333 
      30%        5%        7%        8% 
10.000000 26.666667  3.333333  6.666667 

### How has your organization's lost to care rate changed from last year? Q588

In [363]:
ltcr_change <- table(final_data$Q588)
ltcr_change
ltcr_change_perc <- prop.table(ltcr_change)*100
ltcr_change_perc


Decreased Increased No change    Unsure 
        7         6        16         2 


Decreased Increased No change    Unsure 
22.580645 19.354839 51.612903  6.451613 

### Why do you think your organization's lost to care rate has changed? Q589

In [364]:
### open response???

## Do you communicate the Undetectable = Untransmittable (U=U) message to your patients with HIV? Q604

In [365]:
communicate_uequalsu <- table(final_data$Q604)
communicate_uequalsu
communicate_uequalsu_perc <- prop.table(communicate_uequalsu)*100
communicate_uequalsu_perc


    No Unsure    Yes 
     6      4    218 


       No    Unsure       Yes 
 2.631579  1.754386 95.614035 

## Are you aware of your organization's viral suppression rates? Q605

In [366]:
know_suppression_rates <- table(final_data$Q605)
know_suppression_rates
know_suppression_rates_perc <- prop.table(know_suppression_rates)*100
know_suppression_rates_perc


 No Yes 
137  92 


      No      Yes 
59.82533 40.17467 

### What is your organization's viral suppression rate? Q606

In [367]:
suppression_rate <- table(final_data$Q606)
suppression_rate
suppression_rate_perc <- prop.table(suppression_rate)*100
suppression_rate_perc


                                  100%                                    20% 
                                     1                                      1 
                                   48%                                  6-May 
                                     1                                      1 
                                   60%                                    65% 
                                     1                                      1 
                                   69%                                    70% 
                                     1                                      1 
                                   71%                                    75% 
                                     1                                      1 
                                   76%                                    78% 
                                     1                                      1 
                                   80%             


                                  100%                                    20% 
                              1.098901                               1.098901 
                                   48%                                  6-May 
                              1.098901                               1.098901 
                                   60%                                    65% 
                              1.098901                               1.098901 
                                   69%                                    70% 
                              1.098901                               1.098901 
                                   71%                                    75% 
                              1.098901                               1.098901 
                                   76%                                    78% 
                              1.098901                               1.098901 
                                   80%             

### How has your viral suppression rate changed from last year? Q607

In [368]:
suppression_rate_change <- table(final_data$Q607)
suppression_rate_change
suppression_rate_change_perc <- prop.table(suppression_rate_change)*100
suppression_rate_change_perc


Decreased Increased No change    Unsure 
       10        30        41        11 


Decreased Increased No change    Unsure 
 10.86957  32.60870  44.56522  11.95652 

### Why do you think your organization's viral suppression rate has changed? Q608 

In [369]:
## open response???

## Which component of the HIV care continuum, if any, do you believe needs the most attention in your organization? Q609

In [370]:
hiv_continuum_component <- table(final_data$Q609)
hiv_continuum_component
hiv_continuum_component_perc <- prop.table(hiv_continuum_component)*100
hiv_continuum_component_perc


Initial receipt of care/Starting ART                      Linkage to care 
                                  46                                   89 
                   None of the above                    Retention in Care 
                                  89                                  254 
               Testing and Diagnosis                    Viral suppression 
                                 132                                   51 


Initial receipt of care/Starting ART                      Linkage to care 
                            6.959153                            13.464448 
                   None of the above                    Retention in Care 
                           13.464448                            38.426626 
               Testing and Diagnosis                    Viral suppression 
                           19.969743                             7.715582 

## What do you most need to improve HIV services at your organization? (Select one) Q610

In [371]:
hiv_service_improvement <- table(final_data$Q610)
hiv_service_improvement
hiv_service_improvement_perc <- prop.table(hiv_service_improvement)*100
hiv_service_improvement_perc


                                          Access to HIV related data 
                                                                   5 
                Dedicated insurance or health navigation specialists 
                                                                  25 
                       Fiscal health/funding to support HIV services 
                                                                 161 
              Increased insurance coverage for HIV specific services 
                                                                  36 
                     Increased reimbursement rates or considerations 
                                                                  26 
            Marketing resources to educate the public about HIV care 
                                                                  63 
                             More HIV focused training for providers 
                                                                  44 
                   


                                          Access to HIV related data 
                                                           0.7564297 
                Dedicated insurance or health navigation specialists 
                                                           3.7821483 
                       Fiscal health/funding to support HIV services 
                                                          24.3570348 
              Increased insurance coverage for HIV specific services 
                                                           5.4462935 
                     Increased reimbursement rates or considerations 
                                                           3.9334342 
            Marketing resources to educate the public about HIV care 
                                                           9.5310136 
                             More HIV focused training for providers 
                                                           6.6565809 
                   

## What else do you most need to improve HIV services at your organization? Q611

In [397]:
# open response???

# PrEP Questions

## Do you think client/patient interest in PrEP has changed in the past year? Q791

In [373]:
neg_pol_count <- table(final_data$Q791)
neg_pol_count

neg_pol_perc <- prop.table(neg_pol_count)*100
neg_pol_perc


 No Yes 
203 573 


      No      Yes 
26.15979 73.84021 

### How has interest in PrEP changed from last year? Q672

In [375]:
prep_int_count <- table(final_data$Q672)
prep_int_count

prep_int_perc <- prop.table(prep_int_count)*100
prep_int_perc


Decreased Increased No change    Unsure 
       18       134         1         8 


Decreased Increased No change    Unsure 
11.180124 83.229814  0.621118  4.968944 

## Do you know how many people your organization linked to PrEP last year? Q694

In [376]:
prep_link_aware_count <- table(final_data$Q694)
prep_link_aware_count

prep_link_aware_perc <- prop.table(prep_link_aware_count)*100
prep_link_aware_perc


 No Yes 
180  64 


      No      Yes 
73.77049 26.22951 

### How many people in your organization were linked to PrEP last year? Q695

In [377]:
prep_link_count <- table(final_data$Q695)
prep_link_count

prep_link_perc <- prop.table(prep_link_count)*100
prep_link_perc


     < 50 1000-1999   501-999    51-500 
       36         4         1        23 


     < 50 1000-1999   501-999    51-500 
  56.2500    6.2500    1.5625   35.9375 

### How has the number of people linked to PrEP changed from last year? Q696

In [378]:
prep_change_count <- table(final_data$Q696)
prep_change_count

prep_change_perc <- prop.table(prep_change_count)*100
prep_change_perc


Decreased Increased No change    Unsure 
        3        37        17         7 


Decreased Increased No change    Unsure 
   4.6875   57.8125   26.5625   10.9375 

## How satisfied are you with your current retention in care practices? Q770

In [379]:
ret_sat_count <- table(final_data$Q770)
ret_sat_count

ret_sat_perc <- prop.table(ret_sat_count)*100
ret_sat_perc


     Dissatisfied           Neutral         Satisfied Very dissatisfied 
                9                51                74                 2 
   Very satisfied 
               28 


     Dissatisfied           Neutral         Satisfied Very dissatisfied 
         5.487805         31.097561         45.121951          1.219512 
   Very satisfied 
        17.073171 

## Which stage along the PrEP care continuum, if any, do you believe needs the most attention in your organization? Q771

In [380]:
cont_att_count <- table(final_data$Q771)
cont_att_count

cont_att_perc <- prop.table(cont_att_count)*100
cont_att_perc


      Adherence to PrEP         Linkage to PrEP Outreach/PrEP Promotion 
                     65                      79                     244 
            PrEP Uptake        Prescribing PrEP       Retention in Care 
                     51                      31                     111 


      Adherence to PrEP         Linkage to PrEP Outreach/PrEP Promotion 
              11.187608               13.597246               41.996558 
            PrEP Uptake        Prescribing PrEP       Retention in Care 
               8.777969                5.335628               19.104991 

## How confident are you in prescribing PrEP to transgender individuals and/or people receiving hormone therapy? Q722

In [381]:
conf_pre_count <- table(final_data$Q722)
conf_pre_count

conf_pre_perc <- prop.table(conf_pre_count)*100
conf_pre_perc


    Fairly confident Not confident at all   Slightly confident 
                  19                    8                    1 
  Somewhat confident       Very confident 
                   7                   91 


    Fairly confident Not confident at all   Slightly confident 
          15.0793651            6.3492063            0.7936508 
  Somewhat confident       Very confident 
           5.5555556           72.2222222 

## Do you provide STI testing services (gonorrhea, chlamydia, syphilis) during PrEP appointments? Q723

In [382]:
sti_serv_count <- table(final_data$Q723)
sti_serv_count

sti_serv_perc <- prop.table(sti_serv_count)*100
sti_serv_perc


 No Yes 
 12 114 


      No      Yes 
 9.52381 90.47619 

### How often do you provide STI testing services during PrEP appointments? Q724

In [383]:
sti_freq_count <- table(final_data$Q724)
sti_freq_count

sti_freq_perc <- prop.table(sti_freq_count)*100
sti_freq_perc


Only when symptoms/concerns arise     Routinely/at every PrEP visit 
                                8                               106 


Only when symptoms/concerns arise     Routinely/at every PrEP visit 
                         7.017544                         92.982456 

## Approximately what percentage of your patients rely on patient assistance programs to access PrEP? Q743

In [384]:
assis_perc_count <- table(final_data$Q743)
assis_perc_count

assis_perc_perc <- prop.table(assis_perc_count)*100
assis_perc_perc


               <49%              50-59%              60-69%              70-79% 
                 27                  13                  11                  10 
             80-89%              90-99% None/not applicable              Unsure 
                  8                  23                   6                  28 


               <49%              50-59%              60-69%              70-79% 
          21.428571           10.317460            8.730159            7.936508 
             80-89%              90-99% None/not applicable              Unsure 
           6.349206           18.253968            4.761905           22.222222 

## Do you know how many people your organization prescribed PrEP to last year? Q744

In [385]:
org_pres_aware_count <- table(final_data$Q744)
org_pres_aware_count

org_pres_aware_perc <- prop.table(org_pres_aware_count)*100
org_pres_aware_perc


 No Yes 
 93  33 


      No      Yes 
73.80952 26.19048 

### How has the number of people your organization prescribed PrEP to changed from last year? Q746

In [386]:
org_pres_change_count <- table(final_data$Q746)
org_pres_change_count

org_pres_change_perc <- prop.table(org_pres_change_count)*100
org_pres_change_perc


Decreased Increased No change    Unsure 
        1        23         7         2 


Decreased Increased No change    Unsure 
 3.030303 69.696970 21.212121  6.060606 

## Do you use any retention in care strategies for PrEP select all that apply Q760-768

In [387]:
# Create a vector of column names representing PrEP retention strategies
prep_ret_strats <- c('Q760','Q761','Q762','Q763','Q764','Q765','Q766','Q767','Q768')

In [388]:
# Create a list to store results
prep_ret_strats_results <- list()
# Call the 'get_options_counts_and_percentage' function with 'final_data' and 'prep_ret_stats'
prep_ret_strats_results <- get_option_counts_and_percentage(final_data, prep_ret_strats)
prep_ret_strats_results

$option_counts
Q760 Q761 Q763 Q762 Q766 Q764 Q765 Q768 Q767 
 144  135  126  122   95   94   73    3    0 

$percentages
     Q760      Q761      Q762      Q763      Q764      Q765      Q766      Q767 
86.227545 80.838323 73.053892 75.449102 56.287425 43.712575 56.886228  0.000000 
     Q768 
 1.796407

# Services provided

## Primary care Q15-20

In [389]:
# Create a vector of column names representing primary care services
prim_care_servs <- c('Q15','Q16','Q17','Q18','Q19','Q20')
# Create a list to store results
prim_care_servs_results <- list()
# Call the 'get_options_counts_and_percentage' function with 'final_data' and 'prim_care_servs'
prim_care_servs_results <- get_option_counts_and_percentage(final_data, prim_care_servs)
prim_care_servs_results

$option_counts
Q15 Q17 Q16 Q19 Q18 Q20 
790 699 695 188 186 176 

$percentages
     Q15      Q16      Q17      Q18      Q19      Q20 
71.36405 62.78229 63.14363 16.80217 16.98284 15.89883

## HIV prevention services Q23-28

In [390]:
# Create a vector of column names representing HIV prevention services
hiv_prev_servs <- c('Q23','Q24','Q25','Q26','Q27','Q28')
# Create a list to store results
hiv_prev_servs_results <- list()
# Call the 'get_options_counts_and_percentage' function with 'final_data' and 'hiv_prev_servs'
hiv_prev_servs_results <- get_option_counts_and_percentage(final_data, hiv_prev_servs)
hiv_prev_servs_results

$option_counts
Q27 Q25 Q26 Q23 Q24 Q28 
936 926 914 775 595  85 

$percentages
      Q23       Q24       Q25       Q26       Q27       Q28 
78.046324 59.919436 93.252769 92.044310 94.259819  8.559919

## Sexual and reproductive health services Q33-41

In [391]:
sex_repro_servs <- c('Q33','Q34','Q35','Q36','Q37','Q38','Q39','Q40','Q41')
# Create a list to store results
sex_repro_servs_results <- list()
# Call the 'get_options_counts_and_percentage' function with 'final_data' and 'sex_repro_servs'
sex_repro_servs_results <- get_option_counts_and_percentage(final_data, sex_repro_servs)
sex_repro_servs_results

$option_counts
Q34 Q38 Q37 Q36 Q39 Q33 Q35 Q41 Q40 
881 618 615 582 455 353 332 156  77 

$percentages
      Q33       Q34       Q35       Q36       Q37       Q38       Q39       Q40 
31.859206 79.512635 29.963899 52.527076 55.505415 55.776173 41.064982  6.949458 
      Q41 
14.079422

## Behavioral health Q43-54

In [392]:
# Create a vector of column names representing behavioral health services
behav_health_servs <- c('Q43','Q44','Q45','Q46','Q47','Q48','Q49','Q50','Q51','Q52','Q53','Q54')
# Create a list to store results
behav_health_servs_results <- list()
# Call the 'get_options_counts_and_percentage' function with 'final_data' and 'behav_health_servs'
behav_health_servs_results <- get_option_counts_and_percentage(final_data, behav_health_servs)
behav_health_servs_results

$option_counts
Q43 Q45 Q46 Q44 Q50 Q51 Q49 Q54 Q47 Q52 Q48 Q53 
633 407 391 355 289 267 204 189 145 133  74  70 

$percentages
      Q43       Q44       Q45       Q46       Q47       Q48       Q49       Q50 
57.181572 32.068654 36.766034 35.320687 13.098464  6.684734 18.428184 26.106594 
      Q51       Q52       Q53       Q54 
24.119241 12.014453  6.323397 17.073171

## Social support services Q56-63

In [393]:
# Create a vector of column names representing social support services
soc_supp_servs <- c('Q56','Q57','Q58','Q59','Q60','Q61','Q62','Q63')
# Create a list to store results
soc_supp_servs_results <- list()
# Call the 'get_options_counts_and_percentage' function with 'final_data' and 'soc_supp_servs'
soc_supp_servs_results <- get_option_counts_and_percentage(final_data, soc_supp_servs)
soc_supp_servs_results

$option_counts
Q61 Q56 Q60 Q59 Q58 Q63 Q62 Q57 
832 716 610 552 484 132  75  45 

$percentages
      Q56       Q57       Q58       Q59       Q60       Q61       Q62       Q63 
64.737794  4.068716 43.761302 49.909584 55.153707 75.226040  6.781193 11.934901

## Behavioral screening Q248-254

In [394]:
# Create a vector of column names representing behavioral screening services
behav_screen_servs <- c('Q248','Q249','Q250','Q251','Q252','Q253','Q254')
# Create a list to store results
behav_screen_servs_results <- list()
# Call the 'get_options_counts_and_percentage' function with 'final_data' and 'behav_screen_servs'
behav_screen_servs_results <- get_option_counts_and_percentage(final_data, behav_screen_servs)
behav_screen_servs_results

$option_counts
Q249 Q248 Q251 Q250 Q252 Q253 Q254 
 119  113  113  102   51   30   17 

$percentages
    Q248     Q249     Q250     Q251     Q252     Q253     Q254 
76.87075 80.95238 69.38776 76.87075 34.69388 20.40816 11.56463

## Physical screening Q244-247

In [395]:
# Create a vector of column names representing physical screening services
phys_screen_servs <- c('Q244','Q245','Q246','Q247')
# Create a list to store results
phys_screen_servs_results <- list()
# Call the 'get_options_counts_and_percentage' function with 'final_data' and 'phys_screen_servs'
phys_screen_servs_results <- get_option_counts_and_percentage(final_data, phys_screen_servs)
phys_screen_servs_results

$option_counts
Q245 Q244 Q246 Q247 
 101   95   58   41 

$percentages
    Q244     Q245     Q246     Q247 
64.62585 68.70748 39.45578 27.89116

## Social determinant screening Q255-260

In [396]:
# Create a vector of column names representing SDOH screening services
sdoh_screen_servs <- c('Q255','Q256','Q257','Q258','Q259','Q260')
# Create a list to store results
sdoh_screen_servs_results <- list()
# Call the 'get_options_counts_and_percentage' function with 'final_data' and 'sdoh_screen_servs'
sdoh_screen_servs_results <- get_option_counts_and_percentage(final_data, sdoh_screen_servs)
sdoh_screen_servs_results

$option_counts
Q255 Q256 Q258 Q259 Q257 Q260 
 101  100   99   95   83   18 

$percentages
    Q255     Q256     Q257     Q258     Q259     Q260 
68.70748 68.02721 56.46259 67.34694 64.62585 12.24490